In [1]:
from sklearn.cluster import KMeans
from sklearn.externals import joblib
import numpy as np
from os.path import join
from os import listdir
import cv2
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing import image
from scipy import spatial
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
model = ResNet50(weights='imagenet', include_top=False)

# Zipper Type Shoes : 

In [73]:
images_no = [-1]
images = []
directory = listdir('zipper_data_rgb/')
# directory.pop(0)
kmeans = joblib.load('kmeans_clustering/kmeans_15_resnet_imagenet_zipper')


    
def write_image(best_img,cluster_no):
    key = ['temp']
    value = ['temp']
    data_directory = listdir('data')
    for i in best_img[1:] :
        if type(i)==dict:
            key.append( list(i.keys())[0] )
            value.append( list(i.values())[0] )
        else:
            key.append('temp')
            value.append('temp')
            
    if key.count(cluster_no)>1 : 
        image_name = best_img[0]
        dict_index = [0]
        for i in best_img[1:] :
            if type(i)==dict:
                if (list(i.keys())[0] ==cluster_no) :
                    if dict_index[0] == 0 :
                        index_no = best_img.index(i)
                        value_distance = list( i.values() )[0]
                        dict_index[0] = {index_no:value_distance}
                    
                    else :
                        index_no = best_img.index(i)
                        value_distance = list( i.values() )[0]
                        if list(dict_index[0].values())[0] > value_distance :
                            dict_index[0]={index_no:value_distance}
                                            
        best_image_view = str( list( dict_index[0].keys() )[0] )    
        best_image_name = image_name+'_view_'+best_image_view+'.jpg'
        img = cv2.imread('data/'+best_image_name)
        img = cv2.resize(img , (512,512))
        cv2.imwrite('Best_views/Zipper_best/'+best_image_name,img)
        for k in ['view_1','view_2','view_3','view_4','view_5']:
            full_image_name = image_name+'_'+k+'.jpg'
            if (full_image_name in data_directory and full_image_name!=best_image_name):
                img = cv2.imread('data/'+full_image_name)
                img = cv2.resize(img , (512,512))
                cv2.imwrite('Best_views/Zipper_nobest/'+image_name+'_'+k+'.jpg',img)
                
    else :
        image_name = best_img[0]
        dict_index = [0]
        print (best_img)
        for i in best_img[1:]:
            if type(i) == dict :
                if (list(i.keys())[0] ==cluster_no) :
                    index_no = best_img.index(i)
                    value_distance = list( i.values() )[0]
                    dict_index[0] = {index_no:value_distance}
                    
#         print ("-========",dict_index)
        print (dict_index)
        best_image_view = str( list( dict_index[0].keys() )[0] )    
        best_image_name = image_name+'_view_'+best_image_view+'.jpg'
        img = cv2.imread('data/'+best_image_name)
        img = cv2.resize(img , (512,512))
        cv2.imwrite('Best_views/Zipper_best/'+best_image_name,img)
        for k in ['view_1','view_2','view_3','view_4','view_5']:
            full_image_name = image_name+'_'+k+'.jpg'
            if (full_image_name in data_directory and full_image_name!=best_image_name):
                img = cv2.imread('data/'+full_image_name)
                img = cv2.resize(img , (512,512))
                cv2.imwrite('Best_views/Zipper_nobest/'+image_name+'_'+k+'.jpg',img)
                
            
def write_all_image(best_img):
    data_directory = listdir('data')
    image_name = best_img[0]
    for k in ['view_1','view_2','view_3','view_4','view_5']:
        full_image_name = image_name+'_'+k+'.jpg'
        if (full_image_name in data_directory ):
            img = cv2.imread('data/'+full_image_name)
            img = cv2.resize(img , (512,512))
            cv2.imwrite('Best_views/Zipper_nobest/'+image_name+'_'+k+'.jpg',img)
    

    
def same_image(images):
    images_cmp = []
    best_img = [0]*6
    best_img[0]=images
    for j in ['view_1','view_2','view_3','view_4','view_5']:
        image_path = images+'_'+j+'.jpg'

        if image_path in directory :
            images_cmp.append('zipper_data_rgb/'+image_path)
                
    return images_cmp,best_img


for i in directory :
    item = i.split('_')
    if item[1] == images_no[-1]:
        pass
    else :
        images_no.append(item[1])
        images.append(item[0]+'_'+item[1])


 
    
for k in range(len(images)) :
    print ("---------------------------------")
    images_cmp,best_img = same_image(images[k])
    print (images_cmp)
    for l in images_cmp :
        img = image.load_img(l, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)
        resnet50_feature = model.predict(img_data)
        resnet50_feature = np.array(resnet50_feature.flatten())
        resnet50_feature = np.array(resnet50_feature)
        resnet50_feature = np.expand_dims(resnet50_feature, axis=0)
        #     print (resnet50_feature.shape)
        pred_view = kmeans.predict(resnet50_feature)
        l_index = int(l.split('_')[-1].split('.')[0])

        if pred_view[0] in [3,7,14,11,0,1,6]:
            result = spatial.distance.cosine(kmeans.cluster_centers_[pred_view[0]],resnet50_feature.flatten())
            best_img[l_index] = {pred_view[0]:result}
            

        else :
            result = spatial.distance.cosine(kmeans.cluster_centers_[pred_view[0]],resnet50_feature.flatten())
            best_img[l_index] = {pred_view[0]:result}
            
    
            
    key =[]
    for m in range(1,len(best_img)):

        if (type(best_img[m]) == dict):
            key.append( list(best_img[m].keys())[0] )
            
            
    if 3 in key :
        write_image(best_img,3)
        continue
        
    elif 7 in key :
        write_image(best_img,7)
        continue
        
    elif 14 in key :
        write_image(best_img,14)
        continue
            
    elif 11 in key :
        write_image(best_img,11)
        print ("in 4")
        continue
        
    elif 0 in key :
        write_image(best_img,0)
        continue
            
    elif 1 in key :
        write_image(best_img,1)
        continue
        
    elif 6 in key :
        write_image(best_img,6)
        continue
        
    else :
        print ('here')
        write_all_image(best_img)
        continue
        
    
    


---------------------------------
['zipper_data_rgb/zipper_0_view_1.jpg', 'zipper_data_rgb/zipper_0_view_2.jpg', 'zipper_data_rgb/zipper_0_view_3.jpg', 'zipper_data_rgb/zipper_0_view_4.jpg', 'zipper_data_rgb/zipper_0_view_5.jpg']
['zipper_0', {1: 0.235995352268219}, {5: 0.5732659697532654}, {1: 0.21127080917358398}, {2: 0.2631625533103943}, {3: 0.29320353269577026}]
[{5: 0.29320353269577026}]
---------------------------------
['zipper_data_rgb/zipper_100_view_1.jpg', 'zipper_data_rgb/zipper_100_view_2.jpg', 'zipper_data_rgb/zipper_100_view_3.jpg', 'zipper_data_rgb/zipper_100_view_4.jpg', 'zipper_data_rgb/zipper_100_view_5.jpg']
['zipper_100', {1: 0.18802499771118164}, {12: 0.5326718091964722}, {11: 0.22378838062286377}, {2: 0.24385809898376465}, {3: 0.27995890378952026}]
[{5: 0.27995890378952026}]
---------------------------------
['zipper_data_rgb/zipper_101_view_1.jpg', 'zipper_data_rgb/zipper_101_view_2.jpg', 'zipper_data_rgb/zipper_101_view_3.jpg', 'zipper_data_rgb/zipper_101_view_

---------------------------------
['zipper_data_rgb/zipper_119_view_1.jpg', 'zipper_data_rgb/zipper_119_view_2.jpg', 'zipper_data_rgb/zipper_119_view_3.jpg', 'zipper_data_rgb/zipper_119_view_4.jpg', 'zipper_data_rgb/zipper_119_view_5.jpg']
['zipper_119', {0: 0.33415770530700684}, {12: 0.4567306637763977}, {9: 0.33917397260665894}, {7: 0.2583974599838257}, {4: 0.29033321142196655}]
[{4: 0.2583974599838257}]
---------------------------------
['zipper_data_rgb/zipper_11_view_1.jpg', 'zipper_data_rgb/zipper_11_view_2.jpg', 'zipper_data_rgb/zipper_11_view_3.jpg', 'zipper_data_rgb/zipper_11_view_4.jpg', 'zipper_data_rgb/zipper_11_view_5.jpg']
['zipper_11', {0: 0.2905065417289734}, {12: 0.6390163600444794}, {9: 0.335543155670166}, {2: 0.31029313802719116}, {7: 0.30298399925231934}]
[{5: 0.30298399925231934}]
---------------------------------
['zipper_data_rgb/zipper_120_view_1.jpg', 'zipper_data_rgb/zipper_120_view_2.jpg', 'zipper_data_rgb/zipper_120_view_3.jpg', 'zipper_data_rgb/zipper_120_v

---------------------------------
['zipper_data_rgb/zipper_13_view_1.jpg', 'zipper_data_rgb/zipper_13_view_2.jpg', 'zipper_data_rgb/zipper_13_view_3.jpg', 'zipper_data_rgb/zipper_13_view_4.jpg', 'zipper_data_rgb/zipper_13_view_5.jpg']
['zipper_13', {1: 0.23021793365478516}, {5: 0.3481972813606262}, {11: 0.15413975715637207}, {2: 0.21858912706375122}, {3: 0.22220128774642944}]
[{5: 0.22220128774642944}]
---------------------------------
['zipper_data_rgb/zipper_140_view_1.jpg', 'zipper_data_rgb/zipper_140_view_2.jpg', 'zipper_data_rgb/zipper_140_view_3.jpg', 'zipper_data_rgb/zipper_140_view_4.jpg', 'zipper_data_rgb/zipper_140_view_5.jpg']
['zipper_140', {1: 0.2796863317489624}, {5: 0.48626625537872314}, {1: 0.21163588762283325}, {8: 0.449581503868103}, {3: 0.22186648845672607}]
[{5: 0.22186648845672607}]
---------------------------------
['zipper_data_rgb/zipper_141_view_1.jpg', 'zipper_data_rgb/zipper_141_view_2.jpg', 'zipper_data_rgb/zipper_141_view_3.jpg', 'zipper_data_rgb/zipper_141

['zipper_158', {10: 0.3937866687774658}, {13: 0.4448058605194092}, {4: 0.4691370725631714}, {6: 0.3507486581802368}, {8: 0.43296438455581665}]
[{4: 0.3507486581802368}]
---------------------------------
['zipper_data_rgb/zipper_159_view_1.jpg', 'zipper_data_rgb/zipper_159_view_2.jpg', 'zipper_data_rgb/zipper_159_view_3.jpg', 'zipper_data_rgb/zipper_159_view_4.jpg', 'zipper_data_rgb/zipper_159_view_5.jpg']
---------------------------------
['zipper_data_rgb/zipper_15_view_1.jpg', 'zipper_data_rgb/zipper_15_view_2.jpg', 'zipper_data_rgb/zipper_15_view_3.jpg', 'zipper_data_rgb/zipper_15_view_4.jpg', 'zipper_data_rgb/zipper_15_view_5.jpg']
in 4
---------------------------------
['zipper_data_rgb/zipper_160_view_1.jpg', 'zipper_data_rgb/zipper_160_view_2.jpg', 'zipper_data_rgb/zipper_160_view_3.jpg', 'zipper_data_rgb/zipper_160_view_4.jpg', 'zipper_data_rgb/zipper_160_view_5.jpg']
['zipper_160', {1: 0.35423964262008667}, {5: 0.4144566059112549}, {11: 0.2065156102180481}, {1: 0.2469822168350

['zipper_178', {1: 0.4185406565666199}, {13: 0.4425497055053711}, {8: 0.506606787443161}, {8: 0.6592813730239868}, {3: 0.3644798994064331}]
[{5: 0.3644798994064331}]
---------------------------------
['zipper_data_rgb/zipper_179_view_1.jpg', 'zipper_data_rgb/zipper_179_view_2.jpg', 'zipper_data_rgb/zipper_179_view_3.jpg', 'zipper_data_rgb/zipper_179_view_4.jpg', 'zipper_data_rgb/zipper_179_view_5.jpg']
['zipper_179', {1: 0.17463237047195435}, {5: 0.4822133183479309}, {11: 0.21131974458694458}, {2: 0.24073892831802368}, {3: 0.259904682636261}]
[{5: 0.259904682636261}]
---------------------------------
['zipper_data_rgb/zipper_17_view_1.jpg', 'zipper_data_rgb/zipper_17_view_2.jpg', 'zipper_data_rgb/zipper_17_view_3.jpg', 'zipper_data_rgb/zipper_17_view_4.jpg', 'zipper_data_rgb/zipper_17_view_5.jpg']
['zipper_17', {10: 0.31611430644989014}, {13: 0.36981821060180664}, {4: 0.3799068331718445}, {8: 0.4125705361366272}, {6: 0.3323870897293091}]
[{5: 0.3323870897293091}]
----------------------

...

---------------------------------
['zipper_data_rgb/zipper_457_view_1.jpg', 'zipper_data_rgb/zipper_457_view_2.jpg', 'zipper_data_rgb/zipper_457_view_3.jpg', 'zipper_data_rgb/zipper_457_view_4.jpg', 'zipper_data_rgb/zipper_457_view_5.jpg']
['zipper_457', {0: 0.2627112865447998}, {12: 0.4678524136543274}, {6: 0.2233702540397644}, {4: 0.2664128541946411}, {7: 0.3191617727279663}]
[{5: 0.3191617727279663}]
---------------------------------
['zipper_data_rgb/zipper_458_view_1.jpg', 'zipper_data_rgb/zipper_458_view_2.jpg', 'zipper_data_rgb/zipper_458_view_3.jpg', 'zipper_data_rgb/zipper_458_view_4.jpg', 'zipper_data_rgb/zipper_458_view_5.jpg']
['zipper_458', {0: 0.2614714503288269}, {12: 0.468428373336792}, {9: 0.27442091703414917}, {7: 0.2891693115234375}, {2: 0.2677869200706482}]
[{4: 0.2891693115234375}]
---------------------------------
['zipper_data_rgb/zipper_459_view_1.jpg', 'zipper_data_rgb/zipper_459_view_2.jpg', 'zipper_data_rgb/zipper_459_view_3.jpg', 'zipper_data_rgb/zipper_459_

['zipper_477', {0: 0.3379591703414917}, {12: 0.5103176832199097}, {6: 0.3459939956665039}, {2: 0.33728504180908203}, {7: 0.29470008611679077}]
[{5: 0.29470008611679077}]
---------------------------------
['zipper_data_rgb/zipper_478_view_1.jpg', 'zipper_data_rgb/zipper_478_view_2.jpg', 'zipper_data_rgb/zipper_478_view_3.jpg', 'zipper_data_rgb/zipper_478_view_4.jpg', 'zipper_data_rgb/zipper_478_view_5.jpg']
['zipper_478', {0: 0.27776390314102173}, {12: 0.3974990248680115}, {6: 0.24701058864593506}, {7: 0.22386115789413452}, {4: 0.29246097803115845}]
[{4: 0.22386115789413452}]
---------------------------------
['zipper_data_rgb/zipper_479_view_1.jpg', 'zipper_data_rgb/zipper_479_view_2.jpg', 'zipper_data_rgb/zipper_479_view_3.jpg', 'zipper_data_rgb/zipper_479_view_4.jpg', 'zipper_data_rgb/zipper_479_view_5.jpg']
['zipper_479', {0: 0.28208398818969727}, {12: 0.46571099758148193}, {6: 0.25876086950302124}, {2: 0.3187752366065979}, {7: 0.2370542287826538}]
[{5: 0.2370542287826538}]
--------

['zipper_496', {0: 0.33803820610046387}, {12: 0.45726221799850464}, {9: 0.36020833253860474}, {4: 0.2844657897949219}, {7: 0.349181592464447}]
[{5: 0.349181592464447}]
---------------------------------
['zipper_data_rgb/zipper_497_view_1.jpg', 'zipper_data_rgb/zipper_497_view_2.jpg', 'zipper_data_rgb/zipper_497_view_3.jpg', 'zipper_data_rgb/zipper_497_view_4.jpg', 'zipper_data_rgb/zipper_497_view_5.jpg']
['zipper_497', {10: 0.37012964487075806}, {13: 0.40165555477142334}, {4: 0.3932880163192749}, {8: 0.3879367709159851}, {6: 0.36577099561691284}]
[{5: 0.36577099561691284}]
---------------------------------
['zipper_data_rgb/zipper_498_view_1.jpg', 'zipper_data_rgb/zipper_498_view_2.jpg', 'zipper_data_rgb/zipper_498_view_3.jpg', 'zipper_data_rgb/zipper_498_view_4.jpg', 'zipper_data_rgb/zipper_498_view_5.jpg']
['zipper_498', {1: 0.3043772578239441}, {5: 0.5252573490142822}, {11: 0.2633288502693176}, {2: 0.19836848974227905}, {3: 0.22164028882980347}]
[{5: 0.22164028882980347}]
----------

['zipper_65', {1: 0.3422636389732361}, {8: 0.38113898038864136}, {8: 0.45041710138320923}, {10: 0.4508706331253052}, {8: 0.3676586151123047}]
[{1: 0.3422636389732361}]
---------------------------------
['zipper_data_rgb/zipper_66_view_1.jpg', 'zipper_data_rgb/zipper_66_view_2.jpg', 'zipper_data_rgb/zipper_66_view_3.jpg', 'zipper_data_rgb/zipper_66_view_4.jpg', 'zipper_data_rgb/zipper_66_view_5.jpg']
['zipper_66', {14: 0.38249385356903076}, {8: 0.4371758699417114}, {8: 0.4524463415145874}, {3: 0.4775298237800598}, {8: 0.39215952157974243}]
[{4: 0.4775298237800598}]
---------------------------------
['zipper_data_rgb/zipper_67_view_1.jpg', 'zipper_data_rgb/zipper_67_view_2.jpg', 'zipper_data_rgb/zipper_67_view_3.jpg', 'zipper_data_rgb/zipper_67_view_4.jpg', 'zipper_data_rgb/zipper_67_view_5.jpg']
['zipper_67', {10: 0.56192946434021}, {5: 0.40609341859817505}, {2: 0.36925166845321655}, {8: 0.4191586971282959}, {1: 0.3420652747154236}]
[{5: 0.3420652747154236}]
----------------------------

['zipper_85', {10: 0.5608705878257751}, {5: 0.37738895416259766}, {8: 0.4592215418815613}, {8: 0.47428077459335327}, {11: 0.42590922117233276}]
[{5: 0.42590922117233276}]
in 4
---------------------------------
['zipper_data_rgb/zipper_86_view_1.jpg', 'zipper_data_rgb/zipper_86_view_2.jpg', 'zipper_data_rgb/zipper_86_view_3.jpg', 'zipper_data_rgb/zipper_86_view_4.jpg', 'zipper_data_rgb/zipper_86_view_5.jpg']
['zipper_86', {11: 0.2690301537513733}, {5: 0.4956985116004944}, {11: 0.2660680413246155}, {2: 0.3020622730255127}, {3: 0.34992390871047974}]
[{5: 0.34992390871047974}]
---------------------------------
['zipper_data_rgb/zipper_87_view_1.jpg', 'zipper_data_rgb/zipper_87_view_2.jpg', 'zipper_data_rgb/zipper_87_view_3.jpg', 'zipper_data_rgb/zipper_87_view_4.jpg']
['zipper_87', {0: 0.3428555727005005}, {4: 0.3103649616241455}, {12: 0.4423273801803589}, {9: 0.3040435314178467}, 0]
[{1: 0.3428555727005005}]
---------------------------------
['zipper_data_rgb/zipper_88_view_1.jpg', 'zippe

# Buckle Type Shoes :

In [56]:
images_no = [-1]
images = []
directory = listdir('buckle_rgb/')
directory.pop(0)
kmeans = joblib.load('kmeans_clustering/kmeans_18_resnet_imagenet_buckle')


    
def write_image(best_img,cluster_no):
    key = ['temp']
    value = ['temp']
    data_directory = listdir('data')
    for i in best_img[1:] :
        if type(i)==dict:
            key.append( list(i.keys())[0] )
            value.append( list(i.values())[0] )
        else:
            key.append('temp')
            value.append('temp')
            
    if key.count(cluster_no)>1 : 
        image_name = best_img[0]
        dict_index = [0]
        for i in best_img[1:] :
            if type(i)==dict:
                if (list(i.keys())[0] ==cluster_no) :
                    if dict_index[0] == 0 :
                        index_no = best_img.index(i)
                        value_distance = list( i.values() )[0]
                        dict_index[0] = {index_no:value_distance}
                    
                    else :
                        index_no = best_img.index(i)
                        value_distance = list( i.values() )[0]
                        if list(dict_index[0].values())[0] > value_distance :
                            dict_index[0]={index_no:value_distance}
                                            
        best_image_view = str( list( dict_index[0].keys() )[0] )    
        best_image_name = image_name+'_view_'+best_image_view+'.jpg'
        img = cv2.imread('data/'+best_image_name)
        img = cv2.resize(img , (512,512))
        cv2.imwrite('Best_views/Buckle_best/'+best_image_name,img)
        for k in ['view_1','view_2','view_3','view_4','view_5']:
            full_image_name = image_name+'_'+k+'.jpg'
            if (full_image_name in data_directory and full_image_name!=best_image_name):
                img = cv2.imread('data/'+full_image_name)
                img = cv2.resize(img , (512,512))
                cv2.imwrite('Best_views/Buckle_nobest/'+image_name+'_'+k+'.jpg',img)
                
    else :
        image_name = best_img[0]
        dict_index = [0]
        for i in best_img[1:]:
            if type(i) == dict :
                if (list(i.keys())[0] ==cluster_no) :
                    index_no = best_img.index(i)
                    value_distance = list( i.values() )[0]
                    dict_index[0] = {index_no:value_distance}
                    
#         print ("-========",dict_index)
        best_image_view = str( list( dict_index[0].keys() )[0] )    
        best_image_name = image_name+'_view_'+best_image_view+'.jpg'
        img = cv2.imread('data/'+best_image_name)
        img = cv2.resize(img , (512,512))
        cv2.imwrite('Best_views/Buckle_best/'+best_image_name,img)
        for k in ['view_1','view_2','view_3','view_4','view_5']:
            full_image_name = image_name+'_'+k+'.jpg'
            if (full_image_name in data_directory and full_image_name!=best_image_name):
                img = cv2.imread('data/'+full_image_name)
                img = cv2.resize(img , (512,512))
                cv2.imwrite('Best_views/Buckle_nobest/'+image_name+'_'+k+'.jpg',img)
                
            
def write_all_image(best_img):
    data_directory = listdir('data')
    image_name = best_img[0]
    for k in ['view_1','view_2','view_3','view_4','view_5']:
        full_image_name = image_name+'_'+k+'.jpg'
        if (full_image_name in data_directory ):
            img = cv2.imread('data/'+full_image_name)
            img = cv2.resize(img , (512,512))
            cv2.imwrite('Best_views/Buckle_nobest/'+image_name+'_'+k+'.jpg',img)
    

    
def same_image(images):
    images_cmp = []
    best_img = [0]*6
    best_img[0]=images
    for j in ['view_1','view_2','view_3','view_4','view_5']:
        image_path = images+'_'+j+'.jpg'

        if image_path in directory :
            images_cmp.append('buckle_rgb/'+image_path)
                
    return images_cmp,best_img


for i in directory :
    item = i.split('_')
    if item[1] == images_no[-1]:
        pass
    else :
        images_no.append(item[1])
        images.append(item[0]+'_'+item[1])


 
    
for k in range(len(images)) :
    print ("---------------------------------")
    images_cmp,best_img = same_image(images[k])
    print (images_cmp)
    for l in images_cmp :
        img = image.load_img(l, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)
        resnet50_feature = model.predict(img_data)
        resnet50_feature = np.array(resnet50_feature.flatten())
        resnet50_feature = np.array(resnet50_feature)
        resnet50_feature = np.expand_dims(resnet50_feature, axis=0)
        #     print (resnet50_feature.shape)
        pred_view = kmeans.predict(resnet50_feature)
        l_index = int(l.split('_')[-1].split('.')[0])

        if pred_view[0] in [14,12,7,4]:
            result = spatial.distance.cosine(kmeans.cluster_centers_[pred_view[0]],resnet50_feature.flatten())
            best_img[l_index] = {pred_view[0]:result}
            

        else :
            result = spatial.distance.cosine(kmeans.cluster_centers_[pred_view[0]],resnet50_feature.flatten())
            best_img[l_index] = {pred_view[0]:result}
            
    
            
    key =[]
    for m in range(1,len(best_img)):

        if (type(best_img[m]) == dict):
            key.append( list(best_img[m].keys())[0] )
            
            
    if 14 in key :
        write_image(best_img,14)
        continue
        
    elif 12 in key :
        write_image(best_img,12)
        continue
        
    elif 7 in key :
        write_image(best_img,7)
        continue
        
    elif 4 in key :
        write_image(best_img,4)
        continue
        
    else :
        print ('here')
        write_all_image(best_img)
        continue
        
    
    


---------------------------------
['buckle_rgb/buckle_1713_view_1.jpg', 'buckle_rgb/buckle_1713_view_2.jpg', 'buckle_rgb/buckle_1713_view_3.jpg', 'buckle_rgb/buckle_1713_view_4.jpg', 'buckle_rgb/buckle_1713_view_5.jpg']
---------------------------------
['buckle_rgb/buckle_1714_view_1.jpg', 'buckle_rgb/buckle_1714_view_2.jpg', 'buckle_rgb/buckle_1714_view_3.jpg', 'buckle_rgb/buckle_1714_view_4.jpg', 'buckle_rgb/buckle_1714_view_5.jpg']
---------------------------------
['buckle_rgb/buckle_1715_view_1.jpg', 'buckle_rgb/buckle_1715_view_2.jpg', 'buckle_rgb/buckle_1715_view_3.jpg', 'buckle_rgb/buckle_1715_view_4.jpg', 'buckle_rgb/buckle_1715_view_5.jpg']
---------------------------------
['buckle_rgb/buckle_1716_view_1.jpg', 'buckle_rgb/buckle_1716_view_2.jpg', 'buckle_rgb/buckle_1716_view_3.jpg', 'buckle_rgb/buckle_1716_view_4.jpg', 'buckle_rgb/buckle_1716_view_5.jpg']
---------------------------------
['buckle_rgb/buckle_1717_view_1.jpg', 'buckle_rgb/buckle_1717_view_2.jpg', 'buckle_rgb

---------------------------------
['buckle_rgb/buckle_1751_view_1.jpg', 'buckle_rgb/buckle_1751_view_2.jpg', 'buckle_rgb/buckle_1751_view_3.jpg', 'buckle_rgb/buckle_1751_view_4.jpg', 'buckle_rgb/buckle_1751_view_5.jpg']
---------------------------------
['buckle_rgb/buckle_1752_view_1.jpg', 'buckle_rgb/buckle_1752_view_2.jpg', 'buckle_rgb/buckle_1752_view_3.jpg', 'buckle_rgb/buckle_1752_view_4.jpg', 'buckle_rgb/buckle_1752_view_5.jpg']
---------------------------------
['buckle_rgb/buckle_1753_view_1.jpg', 'buckle_rgb/buckle_1753_view_2.jpg', 'buckle_rgb/buckle_1753_view_3.jpg', 'buckle_rgb/buckle_1753_view_4.jpg', 'buckle_rgb/buckle_1753_view_5.jpg']
---------------------------------
['buckle_rgb/buckle_1754_view_1.jpg', 'buckle_rgb/buckle_1754_view_2.jpg', 'buckle_rgb/buckle_1754_view_3.jpg', 'buckle_rgb/buckle_1754_view_4.jpg', 'buckle_rgb/buckle_1754_view_5.jpg']
---------------------------------
['buckle_rgb/buckle_1755_view_1.jpg', 'buckle_rgb/buckle_1755_view_2.jpg', 'buckle_rgb

---------------------------------
['buckle_rgb/buckle_1789_view_1.jpg', 'buckle_rgb/buckle_1789_view_2.jpg', 'buckle_rgb/buckle_1789_view_3.jpg', 'buckle_rgb/buckle_1789_view_4.jpg', 'buckle_rgb/buckle_1789_view_5.jpg']
---------------------------------
['buckle_rgb/buckle_1790_view_1.jpg', 'buckle_rgb/buckle_1790_view_2.jpg', 'buckle_rgb/buckle_1790_view_3.jpg', 'buckle_rgb/buckle_1790_view_4.jpg', 'buckle_rgb/buckle_1790_view_5.jpg']
---------------------------------
['buckle_rgb/buckle_1791_view_1.jpg', 'buckle_rgb/buckle_1791_view_2.jpg', 'buckle_rgb/buckle_1791_view_3.jpg', 'buckle_rgb/buckle_1791_view_4.jpg', 'buckle_rgb/buckle_1791_view_5.jpg']
---------------------------------
['buckle_rgb/buckle_1792_view_1.jpg', 'buckle_rgb/buckle_1792_view_2.jpg', 'buckle_rgb/buckle_1792_view_3.jpg', 'buckle_rgb/buckle_1792_view_4.jpg', 'buckle_rgb/buckle_1792_view_5.jpg']
---------------------------------
['buckle_rgb/buckle_1793_view_1.jpg', 'buckle_rgb/buckle_1793_view_2.jpg', 'buckle_rgb

---------------------------------
['buckle_rgb/buckle_1827_view_1.jpg', 'buckle_rgb/buckle_1827_view_2.jpg', 'buckle_rgb/buckle_1827_view_3.jpg', 'buckle_rgb/buckle_1827_view_4.jpg', 'buckle_rgb/buckle_1827_view_5.jpg']
---------------------------------
['buckle_rgb/buckle_1828_view_1.jpg', 'buckle_rgb/buckle_1828_view_2.jpg', 'buckle_rgb/buckle_1828_view_3.jpg', 'buckle_rgb/buckle_1828_view_4.jpg', 'buckle_rgb/buckle_1828_view_5.jpg']
---------------------------------
['buckle_rgb/buckle_1829_view_1.jpg', 'buckle_rgb/buckle_1829_view_2.jpg', 'buckle_rgb/buckle_1829_view_3.jpg', 'buckle_rgb/buckle_1829_view_4.jpg', 'buckle_rgb/buckle_1829_view_5.jpg']
---------------------------------
['buckle_rgb/buckle_1830_view_1.jpg', 'buckle_rgb/buckle_1830_view_2.jpg', 'buckle_rgb/buckle_1830_view_3.jpg', 'buckle_rgb/buckle_1830_view_4.jpg', 'buckle_rgb/buckle_1830_view_5.jpg']
---------------------------------
['buckle_rgb/buckle_1831_view_1.jpg', 'buckle_rgb/buckle_1831_view_2.jpg', 'buckle_rgb

# Hook Type Shoes:

In [16]:
images_no = []
images = []
directory = listdir('hook_rgb/')
kmeans = joblib.load('kmeans_12_resnet_imagenet_hook')

def cmp_dist(best_img,idx,result,k):
    
    if best_img[idx] == 0:
        best_img[idx] = {k:result}
    else :
        if list(best_img[idx].items())[0][1]< result :
            best_img[idx] = {k:result}       
        else:
            pass
    return best_img

def write_image(arr,idx,i):
    
    key = list(arr[idx].items())[0][0]
    img_path = key
    img_path = img_path.split('/')[-1]
#     img = cv2.imread('lace_data_rgb/'+img_path)
    img_path = img_path[:4]+'&look'+img_path[4:]
    img = cv2.imread('data/'+img_path)
    img = cv2.resize(img , (512,512))
    cv2.imwrite('Best_views/Hook/'+i+'_'+'.jpg',img)


for i in directory :
    item = i.split('_')
    images_no.append(item[1])
    
    if images_no[-1] in images :
        pass
    else:
        images.append(item[0]+'_'+item[1])

        
for i in images :
    print (i)
    images_cmp = []
    # best_img = 0 * cluster size
    best_img = [0]*12
    for j in ['view_1','view_2','view_3','view_4','view_5']:
        image_path = i+'_'+j+'.jpg'
        
        if image_path in directory :
            images_cmp.append('hook_rgb/'+image_path)
#     print (images_cmp)        
    for k in images_cmp :
        img = image.load_img(k, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)
        resnet50_feature = model.predict(img_data)
        resnet50_feature = np.array(resnet50_feature.flatten())
        resnet50_feature = np.array(resnet50_feature)
        resnet50_feature = np.expand_dims(resnet50_feature, axis=0)
        #     print (resnet50_feature.shape)
        pred_view = kmeans.predict(resnet50_feature)
        k_index = int(k.split('_')[-1].split('.')[0]) - 1
#         print (pred_view)
        if pred_view[0] == 1 :
            result = spatial.distance.cosine(kmeans.cluster_centers_[1],resnet50_feature.flatten())
            best_img = cmp_dist(best_img,pred_view[0],result,k)
            
            
        if pred_view[0] == 8:
            result = spatial.distance.cosine(kmeans.cluster_centers_[8],resnet50_feature.flatten())
            best_img = cmp_dist(best_img,pred_view[0],result,k)
            
        if pred_view[0] == 6 :
            result = spatial.distance.cosine(kmeans.cluster_centers_[6],resnet50_feature.flatten())
            best_img = cmp_dist(best_img,pred_view[0],result,k)
            
        
            
        
            

    for l in best_img :
        if best_img[1] != 0:
            write_image(best_img,1,i)
            break
        
        elif best_img[8] !=0 :
            write_image(best_img,8,i)
            break
            
        elif best_img[6] !=0 :
            write_image(best_img,6,i)
            break
        
        
            
        else :
            print ("No Best Image")
            



hook_1852
hook_1852
hook_1852
hook_1852
hook_1852
hook_1853
hook_1853
hook_1853
hook_1853
hook_1853
hook_1854
hook_1854
hook_1854
hook_1854
hook_1854
hook_1855
hook_1855
hook_1855
hook_1855
hook_1855
hook_1856
hook_1856
hook_1856
hook_1856
hook_1856
hook_1857
hook_1857
hook_1857
hook_1857
hook_1857
hook_1858
hook_1858
hook_1858
hook_1858
hook_1858
hook_1859
hook_1859
hook_1859
hook_1859
hook_1859
hook_1860
hook_1860
hook_1860
hook_1860
hook_1860
hook_1861
hook_1861
hook_1861
hook_1861
hook_1861
hook_1862
hook_1862
hook_1862
hook_1862
hook_1862
hook_1863
hook_1863
hook_1863
hook_1863
hook_1863
hook_1864
hook_1864
hook_1864
hook_1864
hook_1864
hook_1865
hook_1865
hook_1865
hook_1865
hook_1865
hook_1866
hook_1866
hook_1866
hook_1866
hook_1866
hook_1867
hook_1867
hook_1867
hook_1867
hook_1867
hook_1868
hook_1868
hook_1868
hook_1868
hook_1868
hook_1869
hook_1869
hook_1869
hook_1869
hook_1869
hook_1870
hook_1870
hook_1870
hook_1870
hook_1870
hook_1871
hook_1871
hook_1871
hook_1871
hook_1871


hook_2016
hook_2016
hook_2016
hook_2016
hook_2016
hook_2017
hook_2017
hook_2017
hook_2017
hook_2017
hook_2018
hook_2018
hook_2018
hook_2018
hook_2018
hook_2019
hook_2019
hook_2019
hook_2019
hook_2019
hook_2020
hook_2020
hook_2020
hook_2020
hook_2020
hook_2021
hook_2021
hook_2021
hook_2021
hook_2021
hook_2022
hook_2022
hook_2022
hook_2022
hook_2022
hook_2023
hook_2023
hook_2023
hook_2023
hook_2023
hook_2024
hook_2024
hook_2024
hook_2024
hook_2024
hook_2025
hook_2025
hook_2025
hook_2025
hook_2025
hook_2026
hook_2026
hook_2026
hook_2026
hook_2026
hook_2027
hook_2027
hook_2027
hook_2027
hook_2027
hook_2028
hook_2028
hook_2028
hook_2028
hook_2028
hook_2029
hook_2029
hook_2029
hook_2029
hook_2029
hook_2030
hook_2030
hook_2030
hook_2030
hook_2030
hook_2031
hook_2031
hook_2031
hook_2031
hook_2031
hook_2032
hook_2032
hook_2032
hook_2032
hook_2032
hook_2033
hook_2033
hook_2033
hook_2033
hook_2033
hook_2034
hook_2034
hook_2034
hook_2034
hook_2034
hook_2035
hook_2035
hook_2035
hook_2035
hook_2035


In [55]:
images_no = [-1]
images = []
directory = listdir('hook_rgb/')
directory.pop(0)
kmeans = joblib.load('kmeans_clustering/kmeans_12_resnet_imagenet_hook')


    
def write_image(best_img,cluster_no):
    key = ['temp']
    value = ['temp']
    data_directory = listdir('data')
    for i in best_img[1:] :
        if type(i)==dict:
            key.append( list(i.keys())[0] )
            value.append( list(i.values())[0] )
        else:
            key.append('temp')
            value.append('temp')
            
    if key.count(cluster_no)>1 : 
        image_name = best_img[0]
        dict_index = [0]
        for i in best_img[1:] :
            if type(i)==dict:
                if (list(i.keys())[0] ==cluster_no) :
                    if dict_index[0] == 0 :
                        index_no = best_img.index(i)
                        value_distance = list( i.values() )[0]
                        dict_index[0] = {index_no:value_distance}
                    
                    else :
                        index_no = best_img.index(i)
                        value_distance = list( i.values() )[0]
                        if list(dict_index[0].values())[0] > value_distance :
                            dict_index[0]={index_no:value_distance}
                                            
        best_image_view = str( list( dict_index[0].keys() )[0] )    
        best_image_name = image_name+'_view_'+best_image_view+'.jpg'
        img = cv2.imread('data/'+best_image_name)
        img = cv2.resize(img , (512,512))
        cv2.imwrite('Best_views/Hook_best/'+best_image_name,img)
        for k in ['view_1','view_2','view_3','view_4','view_5']:
            full_image_name = image_name+'_'+k+'.jpg'
            if (full_image_name in data_directory and full_image_name!=best_image_name):
                img = cv2.imread('data/'+full_image_name)
                img = cv2.resize(img , (512,512))
                cv2.imwrite('Best_views/Hook_nobest/'+image_name+'_'+k+'.jpg',img)
                
    else :
        image_name = best_img[0]
        dict_index = [0]
        for i in best_img[1:]:
            if type(i) == dict :
                if (list(i.keys())[0] ==cluster_no) :
                    index_no = best_img.index(i)
                    value_distance = list( i.values() )[0]
                    dict_index[0] = {index_no:value_distance}
                    
#         print ("-========",dict_index)
        best_image_view = str( list( dict_index[0].keys() )[0] )    
        best_image_name = image_name+'_view_'+best_image_view+'.jpg'
        img = cv2.imread('data/'+best_image_name)
        img = cv2.resize(img , (512,512))
        cv2.imwrite('Best_views/Hook_best/'+best_image_name,img)
        for k in ['view_1','view_2','view_3','view_4','view_5']:
            full_image_name = image_name+'_'+k+'.jpg'
            if (full_image_name in data_directory and full_image_name!=best_image_name):
                img = cv2.imread('data/'+full_image_name)
                img = cv2.resize(img , (512,512))
                cv2.imwrite('Best_views/Hook_nobest/'+image_name+'_'+k+'.jpg',img)
                
            
def write_all_image(best_img):
    data_directory = listdir('data')
    image_name = best_img[0]
    for k in ['view_1','view_2','view_3','view_4','view_5']:
        full_image_name = image_name+'_'+k+'.jpg'
        if (full_image_name in data_directory ):
            img = cv2.imread('data/'+full_image_name)
            img = cv2.resize(img , (512,512))
            cv2.imwrite('Best_views/Hook_nobest/'+image_name+'_'+k+'.jpg',img)
    

    
def same_image(images):
    images_cmp = []
    best_img = [0]*6
    best_img[0]=images
    for j in ['view_1','view_2','view_3','view_4','view_5']:
        image_path = images+'_'+j+'.jpg'

        if image_path in directory :
            images_cmp.append('hook_rgb/'+image_path)
                
    return images_cmp,best_img


for i in directory :
    item = i.split('_')
    if item[1] == images_no[-1]:
        pass
    else :
        images_no.append(item[1])
        images.append(item[0]+'_'+item[1])


 
    
for k in range(len(images)) :
    print ("---------------------------------")
    images_cmp,best_img = same_image(images[k])
    print (images_cmp)
    for l in images_cmp :
        img = image.load_img(l, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)
        resnet50_feature = model.predict(img_data)
        resnet50_feature = np.array(resnet50_feature.flatten())
        resnet50_feature = np.array(resnet50_feature)
        resnet50_feature = np.expand_dims(resnet50_feature, axis=0)
        #     print (resnet50_feature.shape)
        pred_view = kmeans.predict(resnet50_feature)
        l_index = int(l.split('_')[-1].split('.')[0])

        if pred_view[0] in [1,8,6]:
            result = spatial.distance.cosine(kmeans.cluster_centers_[pred_view[0]],resnet50_feature.flatten())
            best_img[l_index] = {pred_view[0]:result}
            

        else :
            result = spatial.distance.cosine(kmeans.cluster_centers_[pred_view[0]],resnet50_feature.flatten())
            best_img[l_index] = {pred_view[0]:result}
            
    
            
    key =[]
    for m in range(1,len(best_img)):

        if (type(best_img[m]) == dict):
            key.append( list(best_img[m].keys())[0] )
            
            
    if 1 in key :
        write_image(best_img,1)
        continue
        
    elif 8 in key :
        write_image(best_img,8)
        continue
        
    elif 6 in key :
        write_image(best_img,6)
        continue
        
    else :
        print ('here')
        write_all_image(best_img)
        continue
        
    
    


---------------------------------
['hook_rgb/hook_1852_view_2.jpg', 'hook_rgb/hook_1852_view_3.jpg', 'hook_rgb/hook_1852_view_4.jpg', 'hook_rgb/hook_1852_view_5.jpg']
---------------------------------
['hook_rgb/hook_1853_view_1.jpg', 'hook_rgb/hook_1853_view_2.jpg', 'hook_rgb/hook_1853_view_3.jpg', 'hook_rgb/hook_1853_view_4.jpg', 'hook_rgb/hook_1853_view_5.jpg']
---------------------------------
['hook_rgb/hook_1854_view_1.jpg', 'hook_rgb/hook_1854_view_2.jpg', 'hook_rgb/hook_1854_view_3.jpg', 'hook_rgb/hook_1854_view_4.jpg', 'hook_rgb/hook_1854_view_5.jpg']
---------------------------------
['hook_rgb/hook_1855_view_1.jpg', 'hook_rgb/hook_1855_view_2.jpg', 'hook_rgb/hook_1855_view_3.jpg', 'hook_rgb/hook_1855_view_4.jpg', 'hook_rgb/hook_1855_view_5.jpg']
---------------------------------
['hook_rgb/hook_1856_view_1.jpg', 'hook_rgb/hook_1856_view_2.jpg', 'hook_rgb/hook_1856_view_3.jpg', 'hook_rgb/hook_1856_view_4.jpg', 'hook_rgb/hook_1856_view_5.jpg']
---------------------------------

---------------------------------
['hook_rgb/hook_1894_view_1.jpg', 'hook_rgb/hook_1894_view_2.jpg', 'hook_rgb/hook_1894_view_3.jpg', 'hook_rgb/hook_1894_view_4.jpg', 'hook_rgb/hook_1894_view_5.jpg']
---------------------------------
['hook_rgb/hook_1895_view_1.jpg', 'hook_rgb/hook_1895_view_2.jpg', 'hook_rgb/hook_1895_view_3.jpg', 'hook_rgb/hook_1895_view_4.jpg', 'hook_rgb/hook_1895_view_5.jpg']
---------------------------------
['hook_rgb/hook_1896_view_1.jpg', 'hook_rgb/hook_1896_view_2.jpg', 'hook_rgb/hook_1896_view_3.jpg', 'hook_rgb/hook_1896_view_4.jpg', 'hook_rgb/hook_1896_view_5.jpg']
---------------------------------
['hook_rgb/hook_1897_view_1.jpg', 'hook_rgb/hook_1897_view_2.jpg', 'hook_rgb/hook_1897_view_3.jpg', 'hook_rgb/hook_1897_view_4.jpg', 'hook_rgb/hook_1897_view_5.jpg']
---------------------------------
['hook_rgb/hook_1898_view_1.jpg', 'hook_rgb/hook_1898_view_2.jpg', 'hook_rgb/hook_1898_view_3.jpg', 'hook_rgb/hook_1898_view_4.jpg', 'hook_rgb/hook_1898_view_5.jpg']


---------------------------------
['hook_rgb/hook_1935_view_1.jpg', 'hook_rgb/hook_1935_view_2.jpg', 'hook_rgb/hook_1935_view_3.jpg', 'hook_rgb/hook_1935_view_4.jpg', 'hook_rgb/hook_1935_view_5.jpg']
---------------------------------
['hook_rgb/hook_1936_view_1.jpg', 'hook_rgb/hook_1936_view_2.jpg', 'hook_rgb/hook_1936_view_3.jpg', 'hook_rgb/hook_1936_view_4.jpg', 'hook_rgb/hook_1936_view_5.jpg']
---------------------------------
['hook_rgb/hook_1937_view_1.jpg', 'hook_rgb/hook_1937_view_2.jpg', 'hook_rgb/hook_1937_view_3.jpg', 'hook_rgb/hook_1937_view_4.jpg', 'hook_rgb/hook_1937_view_5.jpg']
---------------------------------
['hook_rgb/hook_1938_view_1.jpg', 'hook_rgb/hook_1938_view_2.jpg', 'hook_rgb/hook_1938_view_3.jpg', 'hook_rgb/hook_1938_view_4.jpg', 'hook_rgb/hook_1938_view_5.jpg']
---------------------------------
['hook_rgb/hook_1939_view_1.jpg', 'hook_rgb/hook_1939_view_2.jpg', 'hook_rgb/hook_1939_view_3.jpg', 'hook_rgb/hook_1939_view_4.jpg', 'hook_rgb/hook_1939_view_5.jpg']


---------------------------------
['hook_rgb/hook_1976_view_1.jpg', 'hook_rgb/hook_1976_view_2.jpg', 'hook_rgb/hook_1976_view_3.jpg', 'hook_rgb/hook_1976_view_4.jpg', 'hook_rgb/hook_1976_view_5.jpg']
---------------------------------
['hook_rgb/hook_1977_view_1.jpg', 'hook_rgb/hook_1977_view_2.jpg', 'hook_rgb/hook_1977_view_3.jpg', 'hook_rgb/hook_1977_view_4.jpg', 'hook_rgb/hook_1977_view_5.jpg']
---------------------------------
['hook_rgb/hook_1978_view_1.jpg', 'hook_rgb/hook_1978_view_2.jpg', 'hook_rgb/hook_1978_view_3.jpg', 'hook_rgb/hook_1978_view_4.jpg', 'hook_rgb/hook_1978_view_5.jpg']
---------------------------------
['hook_rgb/hook_1979_view_1.jpg', 'hook_rgb/hook_1979_view_2.jpg', 'hook_rgb/hook_1979_view_3.jpg', 'hook_rgb/hook_1979_view_4.jpg', 'hook_rgb/hook_1979_view_5.jpg']
---------------------------------
['hook_rgb/hook_1980_view_1.jpg', 'hook_rgb/hook_1980_view_2.jpg', 'hook_rgb/hook_1980_view_3.jpg', 'hook_rgb/hook_1980_view_4.jpg', 'hook_rgb/hook_1980_view_5.jpg']


---------------------------------
['hook_rgb/hook_2017_view_1.jpg', 'hook_rgb/hook_2017_view_2.jpg', 'hook_rgb/hook_2017_view_3.jpg', 'hook_rgb/hook_2017_view_4.jpg', 'hook_rgb/hook_2017_view_5.jpg']
---------------------------------
['hook_rgb/hook_2018_view_1.jpg', 'hook_rgb/hook_2018_view_2.jpg', 'hook_rgb/hook_2018_view_3.jpg', 'hook_rgb/hook_2018_view_4.jpg', 'hook_rgb/hook_2018_view_5.jpg']
---------------------------------
['hook_rgb/hook_2019_view_1.jpg', 'hook_rgb/hook_2019_view_2.jpg', 'hook_rgb/hook_2019_view_3.jpg', 'hook_rgb/hook_2019_view_4.jpg', 'hook_rgb/hook_2019_view_5.jpg']
---------------------------------
['hook_rgb/hook_2020_view_1.jpg', 'hook_rgb/hook_2020_view_2.jpg', 'hook_rgb/hook_2020_view_3.jpg', 'hook_rgb/hook_2020_view_4.jpg', 'hook_rgb/hook_2020_view_5.jpg']
---------------------------------
['hook_rgb/hook_2021_view_1.jpg', 'hook_rgb/hook_2021_view_2.jpg', 'hook_rgb/hook_2021_view_3.jpg', 'hook_rgb/hook_2021_view_4.jpg', 'hook_rgb/hook_2021_view_5.jpg']


---------------------------------
['hook_rgb/hook_2058_view_1.jpg', 'hook_rgb/hook_2058_view_2.jpg', 'hook_rgb/hook_2058_view_3.jpg', 'hook_rgb/hook_2058_view_4.jpg', 'hook_rgb/hook_2058_view_5.jpg']
---------------------------------
['hook_rgb/hook_2059_view_1.jpg', 'hook_rgb/hook_2059_view_2.jpg', 'hook_rgb/hook_2059_view_3.jpg', 'hook_rgb/hook_2059_view_4.jpg', 'hook_rgb/hook_2059_view_5.jpg']
---------------------------------
['hook_rgb/hook_2060_view_1.jpg', 'hook_rgb/hook_2060_view_2.jpg', 'hook_rgb/hook_2060_view_3.jpg', 'hook_rgb/hook_2060_view_4.jpg', 'hook_rgb/hook_2060_view_5.jpg']
---------------------------------
['hook_rgb/hook_2061_view_1.jpg', 'hook_rgb/hook_2061_view_2.jpg', 'hook_rgb/hook_2061_view_3.jpg', 'hook_rgb/hook_2061_view_4.jpg', 'hook_rgb/hook_2061_view_5.jpg']
---------------------------------
['hook_rgb/hook_2062_view_1.jpg', 'hook_rgb/hook_2062_view_2.jpg', 'hook_rgb/hook_2062_view_3.jpg', 'hook_rgb/hook_2062_view_4.jpg', 'hook_rgb/hook_2062_view_5.jpg']


---------------------------------
['hook_rgb/hook_2100_view_1.jpg', 'hook_rgb/hook_2100_view_2.jpg', 'hook_rgb/hook_2100_view_3.jpg', 'hook_rgb/hook_2100_view_4.jpg', 'hook_rgb/hook_2100_view_5.jpg']
---------------------------------
['hook_rgb/hook_2101_view_1.jpg', 'hook_rgb/hook_2101_view_2.jpg', 'hook_rgb/hook_2101_view_3.jpg', 'hook_rgb/hook_2101_view_4.jpg', 'hook_rgb/hook_2101_view_5.jpg']
---------------------------------
['hook_rgb/hook_2102_view_1.jpg', 'hook_rgb/hook_2102_view_2.jpg', 'hook_rgb/hook_2102_view_3.jpg', 'hook_rgb/hook_2102_view_4.jpg', 'hook_rgb/hook_2102_view_5.jpg']
---------------------------------
['hook_rgb/hook_2103_view_1.jpg', 'hook_rgb/hook_2103_view_2.jpg', 'hook_rgb/hook_2103_view_3.jpg', 'hook_rgb/hook_2103_view_4.jpg', 'hook_rgb/hook_2103_view_5.jpg']
---------------------------------
['hook_rgb/hook_2104_view_1.jpg', 'hook_rgb/hook_2104_view_2.jpg', 'hook_rgb/hook_2104_view_3.jpg', 'hook_rgb/hook_2104_view_4.jpg', 'hook_rgb/hook_2104_view_5.jpg']


---------------------------------
['hook_rgb/hook_2142_view_1.jpg', 'hook_rgb/hook_2142_view_2.jpg', 'hook_rgb/hook_2142_view_3.jpg', 'hook_rgb/hook_2142_view_4.jpg', 'hook_rgb/hook_2142_view_5.jpg']
---------------------------------
['hook_rgb/hook_2143_view_1.jpg', 'hook_rgb/hook_2143_view_2.jpg', 'hook_rgb/hook_2143_view_3.jpg', 'hook_rgb/hook_2143_view_4.jpg', 'hook_rgb/hook_2143_view_5.jpg']
---------------------------------
['hook_rgb/hook_2144_view_1.jpg', 'hook_rgb/hook_2144_view_2.jpg', 'hook_rgb/hook_2144_view_3.jpg', 'hook_rgb/hook_2144_view_4.jpg', 'hook_rgb/hook_2144_view_5.jpg']
---------------------------------
['hook_rgb/hook_2145_view_1.jpg', 'hook_rgb/hook_2145_view_2.jpg', 'hook_rgb/hook_2145_view_3.jpg', 'hook_rgb/hook_2145_view_4.jpg', 'hook_rgb/hook_2145_view_5.jpg']
---------------------------------
['hook_rgb/hook_2146_view_1.jpg', 'hook_rgb/hook_2146_view_2.jpg', 'hook_rgb/hook_2146_view_3.jpg', 'hook_rgb/hook_2146_view_4.jpg', 'hook_rgb/hook_2146_view_5.jpg']


# Slip-on Based Shoes :

In [5]:
images_no = []
images = []
directory = listdir('slip_data_rgb/')
kmeans = joblib.load('kmeans_9_resnet_imagenet_slipon')

def cmp_dist(best_img,idx,result,k):
    
    if best_img[idx] == 0:
        best_img[idx] = {k:result}
    else :
        if list(best_img[idx].items())[0][1]< result :
            best_img[idx] = {k:result}       
        else:
            pass
    return best_img

def write_image(arr,idx,i):
    
    key = list(arr[idx].items())[0][0]
    img_path = key
    img_path = img_path.split('/')[-1]
    img = cv2.imread('slip_data_rgb/'+img_path)
    cv2.imwrite('Best_views/Slip/'+i+'_'+'.jpg',img)


for i in directory :
    item = i.split('_')
    images_no.append(item[1])
    
    if images_no[-1] in images :
        pass
    else:
        images.append(item[0]+'_'+item[1])

        
for i in images :
    print (i)
    images_cmp = []
    # best_img = 0 * cluster size
    best_img = [0]*9
    for j in ['view_1','view_2','view_3','view_4','view_5']:
        image_path = i+'_'+j+'.jpg'
        
        if image_path in directory :
            images_cmp.append('slip_data_rgb/'+image_path)
#     print (images_cmp)        
    for k in images_cmp :
        img = image.load_img(k, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)
        resnet50_feature = model.predict(img_data)
        resnet50_feature = np.array(resnet50_feature.flatten())
        resnet50_feature = np.array(resnet50_feature)
        resnet50_feature = np.expand_dims(resnet50_feature, axis=0)
        #     print (resnet50_feature.shape)
        pred_view = kmeans.predict(resnet50_feature)
        k_index = int(k.split('_')[-1].split('.')[0]) - 1
#         print (pred_view)
        if pred_view[0] == 2 :
            result = spatial.distance.cosine(kmeans.cluster_centers_[2],resnet50_feature.flatten())
            best_img = cmp_dist(best_img,pred_view[0],result,k)
            
            
        if pred_view[0] == 0 :
            result = spatial.distance.cosine(kmeans.cluster_centers_[0],resnet50_feature.flatten())
            best_img = cmp_dist(best_img,pred_view[0],result,k)
            
        
            

    for l in best_img :
        if best_img[2] != 0:
            write_image(best_img,2,i)
            break
        
        elif best_img[0] !=0 :
            write_image(best_img,0,i)
            break
            
        else :
            print ("No Best Image")
            



slip_1000
slip_1000
slip_1001
slip_1001
slip_1001
slip_1001
slip_1001
slip_1002
slip_1002
slip_1002
slip_1002
slip_1002
slip_1003
slip_1003
slip_1003
slip_1003
slip_1003
slip_1004
slip_1004
slip_1004
slip_1004
slip_1004
slip_1005
slip_1005
slip_1005
slip_1005
slip_1005
slip_1006
slip_1006
slip_1006
slip_1006
slip_1006
slip_1007
slip_1007
slip_1007
slip_1007
slip_1007
slip_1008
slip_1008
slip_1008
slip_1008
slip_1008
slip_1009
slip_1009
slip_1009
slip_1009
slip_1009
slip_1010
slip_1010
slip_1010
slip_1010
slip_1010
slip_1011
slip_1011
slip_1011
slip_1011
slip_1011
slip_1012
slip_1012
slip_1012
slip_1012
slip_1012
slip_1013
slip_1013
slip_1013
slip_1013
slip_1013
slip_1014
slip_1014
slip_1014
slip_1014
slip_1014
slip_1015
slip_1015
slip_1015
slip_1015
slip_1015
slip_1016
slip_1016
slip_1016
slip_1016
slip_1016
slip_1017
slip_1017
slip_1017
slip_1017
slip_1017
slip_1018
slip_1018
slip_1018
slip_1018
slip_1018
slip_1019
slip_1019
slip_1019
slip_1019
slip_1019
slip_1020
slip_1020
slip_1020


slip_1152
slip_1152
slip_1152
slip_1152
slip_1153
slip_1153
slip_1153
slip_1153
slip_1153
slip_1154
slip_1154
slip_1154
slip_1154
slip_1154
slip_1155
slip_1155
slip_1155
slip_1155
slip_1155
slip_1156
slip_1156
slip_1156
slip_1156
slip_1156
slip_1157
slip_1157
slip_1157
slip_1157
slip_1157
slip_1158
slip_1158
slip_1158
slip_1158
slip_1158
slip_1159
slip_1159
slip_1159
slip_1159
slip_1159
slip_1160
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
slip_1160
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
slip_1160
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
slip_1160
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
slip_1160
No Best Image
No Best Image
No Best Image
No Best Image


slip_857
slip_857
slip_857
slip_857
slip_858
slip_858
slip_858
slip_858
slip_858
slip_859
slip_859
slip_859
slip_859
slip_859
slip_860
slip_860
slip_860
slip_860
slip_860
slip_861
slip_861
slip_861
slip_861
slip_861
slip_862
slip_862
slip_862
slip_862
slip_862
slip_863
slip_863
slip_863
slip_863
slip_863
slip_864
slip_864
slip_864
slip_864
slip_864
slip_865
slip_865
slip_865
slip_865
slip_865
slip_866
slip_866
slip_866
slip_866
slip_866
slip_867
slip_867
slip_867
slip_867
slip_867
slip_868
slip_868
slip_868
slip_868
slip_868
slip_869
slip_869
slip_869
slip_869
slip_869
slip_870
slip_870
slip_870
slip_870
slip_870
slip_871
slip_871
slip_871
slip_871
slip_871
slip_872
slip_872
slip_872
slip_872
slip_872
slip_873
slip_873
slip_873
slip_873
slip_873
slip_874
slip_874
slip_874
slip_874
slip_874
slip_875
slip_875
slip_875
slip_875
slip_875
slip_876
slip_876
slip_876
slip_876
slip_876
slip_877
slip_877
slip_877
slip_877
slip_877
slip_878
slip_878
slip_878
slip_878
slip_878
slip_879
slip_879
s

In [53]:
images_no = [-1]
images = []
directory = listdir('slip_rgb/')
directory.pop(0)
kmeans = joblib.load('kmeans_clustering/kmeans_9_resnet_imagenet_slipon')


    
def write_image(best_img,cluster_no):
    key = ['temp']
    value = ['temp']
    data_directory = listdir('data')
    for i in best_img[1:] :
        if type(i)==dict:
            key.append( list(i.keys())[0] )
            value.append( list(i.values())[0] )
        else:
            key.append('temp')
            value.append('temp')
            
    if key.count(cluster_no)>1 : 
        image_name = best_img[0]
        dict_index = [0]
        for i in best_img[1:] :
            if type(i)==dict:
                if (list(i.keys())[0] ==cluster_no) :
                    if dict_index[0] == 0 :
                        index_no = best_img.index(i)
                        value_distance = list( i.values() )[0]
                        dict_index[0] = {index_no:value_distance}
                    
                    else :
                        index_no = best_img.index(i)
                        value_distance = list( i.values() )[0]
                        if list(dict_index[0].values())[0] > value_distance :
                            dict_index[0]={index_no:value_distance}
                                            
        best_image_view = str( list( dict_index[0].keys() )[0] )    
        best_image_name = image_name+'_view_'+best_image_view+'.jpg'
        img = cv2.imread('data/'+best_image_name)
        img = cv2.resize(img , (512,512))
        cv2.imwrite('Best_views/Slip_best/'+best_image_name,img)
        for k in ['view_1','view_2','view_3','view_4','view_5']:
            full_image_name = image_name+'_'+k+'.jpg'
            if (full_image_name in data_directory and full_image_name!=best_image_name):
                img = cv2.imread('data/'+full_image_name)
                img = cv2.resize(img , (512,512))
                cv2.imwrite('Best_views/Slip_nobest/'+image_name+'_'+k+'.jpg',img)
                
    else :
        image_name = best_img[0]
        dict_index = [0]
        for i in best_img[1:]:
            if type(i) == dict :
                if (list(i.keys())[0] ==cluster_no) :
                    index_no = best_img.index(i)
                    value_distance = list( i.values() )[0]
                    dict_index[0] = {index_no:value_distance}
                    
#         print ("-========",dict_index)
        best_image_view = str( list( dict_index[0].keys() )[0] )    
        best_image_name = image_name+'_view_'+best_image_view+'.jpg'
        img = cv2.imread('data/'+best_image_name)
        img = cv2.resize(img , (512,512))
        cv2.imwrite('Best_views/Slip_best/'+best_image_name,img)
        for k in ['view_1','view_2','view_3','view_4','view_5']:
            full_image_name = image_name+'_'+k+'.jpg'
            if (full_image_name in data_directory and full_image_name!=best_image_name):
                img = cv2.imread('data/'+full_image_name)
                img = cv2.resize(img , (512,512))
                cv2.imwrite('Best_views/Slip_nobest/'+image_name+'_'+k+'.jpg',img)
                
            
def write_all_image(best_img):
    data_directory = listdir('data')
    image_name = best_img[0]
    for k in ['view_1','view_2','view_3','view_4','view_5']:
        full_image_name = image_name+'_'+k+'.jpg'
        if (full_image_name in data_directory ):
            img = cv2.imread('data/'+full_image_name)
            img = cv2.resize(img , (512,512))
            cv2.imwrite('Best_views/Slip_nobest/'+image_name+'_'+k+'.jpg',img)
    

    
def same_image(images):
    images_cmp = []
    best_img = [0]*6
    best_img[0]=images
    for j in ['view_1','view_2','view_3','view_4','view_5']:
        image_path = images+'_'+j+'.jpg'

        if image_path in directory :
            images_cmp.append('slip_rgb/'+image_path)
                
    return images_cmp,best_img


for i in directory :
    item = i.split('_')
    if item[1] == images_no[-1]:
        pass
    else :
        images_no.append(item[1])
        images.append(item[0]+'_'+item[1])


 
    
for k in range(len(images)) :
    print ("---------------------------------")
    images_cmp,best_img = same_image(images[k])
    print (images_cmp)
    for l in images_cmp :
        img = image.load_img(l, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)
        resnet50_feature = model.predict(img_data)
        resnet50_feature = np.array(resnet50_feature.flatten())
        resnet50_feature = np.array(resnet50_feature)
        resnet50_feature = np.expand_dims(resnet50_feature, axis=0)
        #     print (resnet50_feature.shape)
        pred_view = kmeans.predict(resnet50_feature)
        l_index = int(l.split('_')[-1].split('.')[0])

        if pred_view[0] in [2,0]:
            result = spatial.distance.cosine(kmeans.cluster_centers_[pred_view[0]],resnet50_feature.flatten())
            best_img[l_index] = {pred_view[0]:result}
            

        else :
            result = spatial.distance.cosine(kmeans.cluster_centers_[pred_view[0]],resnet50_feature.flatten())
            best_img[l_index] = {pred_view[0]:result}
            
    
            
    key =[]
    for m in range(1,len(best_img)):

        if (type(best_img[m]) == dict):
            key.append( list(best_img[m].keys())[0] )
            
            
    if 2 in key :
        write_image(best_img,2)
        continue
        
    elif 0 in key :
        write_image(best_img,0)
        continue
        
    else :
        print ('here')
        write_all_image(best_img)
        continue
        
    
    


---------------------------------
['slip_rgb/slip_1000_view_1.jpg', 'slip_rgb/slip_1000_view_2.jpg']
---------------------------------
['slip_rgb/slip_1001_view_1.jpg', 'slip_rgb/slip_1001_view_2.jpg', 'slip_rgb/slip_1001_view_3.jpg', 'slip_rgb/slip_1001_view_4.jpg', 'slip_rgb/slip_1001_view_5.jpg']
---------------------------------
['slip_rgb/slip_1002_view_1.jpg', 'slip_rgb/slip_1002_view_2.jpg', 'slip_rgb/slip_1002_view_3.jpg', 'slip_rgb/slip_1002_view_4.jpg', 'slip_rgb/slip_1002_view_5.jpg']
---------------------------------
['slip_rgb/slip_1003_view_1.jpg', 'slip_rgb/slip_1003_view_2.jpg', 'slip_rgb/slip_1003_view_3.jpg', 'slip_rgb/slip_1003_view_4.jpg', 'slip_rgb/slip_1003_view_5.jpg']
---------------------------------
['slip_rgb/slip_1004_view_1.jpg', 'slip_rgb/slip_1004_view_2.jpg', 'slip_rgb/slip_1004_view_3.jpg', 'slip_rgb/slip_1004_view_4.jpg', 'slip_rgb/slip_1004_view_5.jpg']
---------------------------------
['slip_rgb/slip_1005_view_1.jpg', 'slip_rgb/slip_1005_view_2.jpg'

---------------------------------
['slip_rgb/slip_1042_view_1.jpg', 'slip_rgb/slip_1042_view_2.jpg', 'slip_rgb/slip_1042_view_3.jpg', 'slip_rgb/slip_1042_view_4.jpg', 'slip_rgb/slip_1042_view_5.jpg']
---------------------------------
['slip_rgb/slip_1043_view_1.jpg', 'slip_rgb/slip_1043_view_2.jpg', 'slip_rgb/slip_1043_view_3.jpg', 'slip_rgb/slip_1043_view_4.jpg', 'slip_rgb/slip_1043_view_5.jpg']
---------------------------------
['slip_rgb/slip_1044_view_1.jpg', 'slip_rgb/slip_1044_view_2.jpg', 'slip_rgb/slip_1044_view_3.jpg', 'slip_rgb/slip_1044_view_4.jpg', 'slip_rgb/slip_1044_view_5.jpg']
---------------------------------
['slip_rgb/slip_1045_view_1.jpg', 'slip_rgb/slip_1045_view_2.jpg', 'slip_rgb/slip_1045_view_3.jpg', 'slip_rgb/slip_1045_view_4.jpg', 'slip_rgb/slip_1045_view_5.jpg']
---------------------------------
['slip_rgb/slip_1046_view_1.jpg', 'slip_rgb/slip_1046_view_2.jpg', 'slip_rgb/slip_1046_view_3.jpg', 'slip_rgb/slip_1046_view_4.jpg', 'slip_rgb/slip_1046_view_5.jpg']


---------------------------------
['slip_rgb/slip_1084_view_1.jpg', 'slip_rgb/slip_1084_view_2.jpg', 'slip_rgb/slip_1084_view_3.jpg', 'slip_rgb/slip_1084_view_4.jpg', 'slip_rgb/slip_1084_view_5.jpg']
---------------------------------
['slip_rgb/slip_1085_view_1.jpg', 'slip_rgb/slip_1085_view_2.jpg', 'slip_rgb/slip_1085_view_3.jpg', 'slip_rgb/slip_1085_view_4.jpg', 'slip_rgb/slip_1085_view_5.jpg']
---------------------------------
['slip_rgb/slip_1086_view_1.jpg', 'slip_rgb/slip_1086_view_2.jpg', 'slip_rgb/slip_1086_view_3.jpg', 'slip_rgb/slip_1086_view_4.jpg', 'slip_rgb/slip_1086_view_5.jpg']
---------------------------------
['slip_rgb/slip_1087_view_1.jpg', 'slip_rgb/slip_1087_view_2.jpg', 'slip_rgb/slip_1087_view_3.jpg', 'slip_rgb/slip_1087_view_4.jpg', 'slip_rgb/slip_1087_view_5.jpg']
---------------------------------
['slip_rgb/slip_1088_view_1.jpg', 'slip_rgb/slip_1088_view_2.jpg', 'slip_rgb/slip_1088_view_3.jpg', 'slip_rgb/slip_1088_view_4.jpg', 'slip_rgb/slip_1088_view_5.jpg']


---------------------------------
['slip_rgb/slip_1125_view_1.jpg', 'slip_rgb/slip_1125_view_2.jpg', 'slip_rgb/slip_1125_view_3.jpg', 'slip_rgb/slip_1125_view_4.jpg', 'slip_rgb/slip_1125_view_5.jpg']
---------------------------------
['slip_rgb/slip_1126_view_1.jpg', 'slip_rgb/slip_1126_view_2.jpg', 'slip_rgb/slip_1126_view_3.jpg', 'slip_rgb/slip_1126_view_4.jpg', 'slip_rgb/slip_1126_view_5.jpg']
---------------------------------
['slip_rgb/slip_1127_view_1.jpg', 'slip_rgb/slip_1127_view_2.jpg', 'slip_rgb/slip_1127_view_3.jpg', 'slip_rgb/slip_1127_view_4.jpg', 'slip_rgb/slip_1127_view_5.jpg']
---------------------------------
['slip_rgb/slip_1128_view_1.jpg', 'slip_rgb/slip_1128_view_2.jpg', 'slip_rgb/slip_1128_view_3.jpg', 'slip_rgb/slip_1128_view_4.jpg', 'slip_rgb/slip_1128_view_5.jpg']
---------------------------------
['slip_rgb/slip_1129_view_1.jpg', 'slip_rgb/slip_1129_view_2.jpg', 'slip_rgb/slip_1129_view_3.jpg', 'slip_rgb/slip_1129_view_4.jpg', 'slip_rgb/slip_1129_view_5.jpg']


---------------------------------
['slip_rgb/slip_1166_view_1.jpg', 'slip_rgb/slip_1166_view_2.jpg', 'slip_rgb/slip_1166_view_3.jpg', 'slip_rgb/slip_1166_view_4.jpg', 'slip_rgb/slip_1166_view_5.jpg']
---------------------------------
['slip_rgb/slip_1167_view_1.jpg', 'slip_rgb/slip_1167_view_2.jpg', 'slip_rgb/slip_1167_view_3.jpg', 'slip_rgb/slip_1167_view_5.jpg']
---------------------------------
['slip_rgb/slip_1168_view_1.jpg', 'slip_rgb/slip_1168_view_2.jpg', 'slip_rgb/slip_1168_view_3.jpg', 'slip_rgb/slip_1168_view_4.jpg', 'slip_rgb/slip_1168_view_5.jpg']
---------------------------------
['slip_rgb/slip_1169_view_1.jpg', 'slip_rgb/slip_1169_view_2.jpg', 'slip_rgb/slip_1169_view_3.jpg', 'slip_rgb/slip_1169_view_4.jpg', 'slip_rgb/slip_1169_view_5.jpg']
---------------------------------
['slip_rgb/slip_1170_view_1.jpg', 'slip_rgb/slip_1170_view_2.jpg', 'slip_rgb/slip_1170_view_3.jpg', 'slip_rgb/slip_1170_view_4.jpg', 'slip_rgb/slip_1170_view_5.jpg']
---------------------------------

...

---------------------------------
['slip_rgb/slip_1249_view_1.jpg', 'slip_rgb/slip_1249_view_2.jpg', 'slip_rgb/slip_1249_view_3.jpg', 'slip_rgb/slip_1249_view_4.jpg', 'slip_rgb/slip_1249_view_5.jpg']
---------------------------------
['slip_rgb/slip_1250_view_1.jpg', 'slip_rgb/slip_1250_view_2.jpg', 'slip_rgb/slip_1250_view_3.jpg', 'slip_rgb/slip_1250_view_4.jpg', 'slip_rgb/slip_1250_view_5.jpg']
---------------------------------
['slip_rgb/slip_1251_view_1.jpg', 'slip_rgb/slip_1251_view_2.jpg', 'slip_rgb/slip_1251_view_3.jpg', 'slip_rgb/slip_1251_view_4.jpg', 'slip_rgb/slip_1251_view_5.jpg']
---------------------------------
['slip_rgb/slip_1252_view_1.jpg', 'slip_rgb/slip_1252_view_2.jpg', 'slip_rgb/slip_1252_view_3.jpg', 'slip_rgb/slip_1252_view_4.jpg', 'slip_rgb/slip_1252_view_5.jpg']
---------------------------------
['slip_rgb/slip_1253_view_1.jpg', 'slip_rgb/slip_1253_view_2.jpg', 'slip_rgb/slip_1253_view_3.jpg', 'slip_rgb/slip_1253_view_4.jpg', 'slip_rgb/slip_1253_view_5.jpg']


---------------------------------
['slip_rgb/slip_866_view_1.jpg', 'slip_rgb/slip_866_view_2.jpg', 'slip_rgb/slip_866_view_3.jpg', 'slip_rgb/slip_866_view_4.jpg', 'slip_rgb/slip_866_view_5.jpg']
---------------------------------
['slip_rgb/slip_867_view_1.jpg', 'slip_rgb/slip_867_view_2.jpg', 'slip_rgb/slip_867_view_3.jpg', 'slip_rgb/slip_867_view_4.jpg', 'slip_rgb/slip_867_view_5.jpg']
---------------------------------
['slip_rgb/slip_868_view_1.jpg', 'slip_rgb/slip_868_view_2.jpg', 'slip_rgb/slip_868_view_3.jpg', 'slip_rgb/slip_868_view_4.jpg', 'slip_rgb/slip_868_view_5.jpg']
---------------------------------
['slip_rgb/slip_869_view_1.jpg', 'slip_rgb/slip_869_view_2.jpg', 'slip_rgb/slip_869_view_3.jpg', 'slip_rgb/slip_869_view_4.jpg', 'slip_rgb/slip_869_view_5.jpg']
---------------------------------
['slip_rgb/slip_870_view_1.jpg', 'slip_rgb/slip_870_view_2.jpg', 'slip_rgb/slip_870_view_3.jpg', 'slip_rgb/slip_870_view_4.jpg', 'slip_rgb/slip_870_view_5.jpg']
-------------------------

---------------------------------
['slip_rgb/slip_909_view_1.jpg', 'slip_rgb/slip_909_view_2.jpg', 'slip_rgb/slip_909_view_3.jpg', 'slip_rgb/slip_909_view_4.jpg', 'slip_rgb/slip_909_view_5.jpg']
---------------------------------
['slip_rgb/slip_910_view_1.jpg', 'slip_rgb/slip_910_view_2.jpg', 'slip_rgb/slip_910_view_3.jpg', 'slip_rgb/slip_910_view_4.jpg', 'slip_rgb/slip_910_view_5.jpg']
---------------------------------
['slip_rgb/slip_911_view_1.jpg', 'slip_rgb/slip_911_view_2.jpg', 'slip_rgb/slip_911_view_3.jpg', 'slip_rgb/slip_911_view_4.jpg', 'slip_rgb/slip_911_view_5.jpg']
---------------------------------
['slip_rgb/slip_912_view_1.jpg', 'slip_rgb/slip_912_view_2.jpg', 'slip_rgb/slip_912_view_3.jpg', 'slip_rgb/slip_912_view_4.jpg', 'slip_rgb/slip_912_view_5.jpg']
---------------------------------
['slip_rgb/slip_913_view_1.jpg', 'slip_rgb/slip_913_view_2.jpg', 'slip_rgb/slip_913_view_3.jpg', 'slip_rgb/slip_913_view_4.jpg', 'slip_rgb/slip_913_view_5.jpg']
-------------------------

---------------------------------
['slip_rgb/slip_951_view_1.jpg', 'slip_rgb/slip_951_view_2.jpg', 'slip_rgb/slip_951_view_3.jpg', 'slip_rgb/slip_951_view_4.jpg', 'slip_rgb/slip_951_view_5.jpg']
---------------------------------
['slip_rgb/slip_952_view_1.jpg', 'slip_rgb/slip_952_view_2.jpg', 'slip_rgb/slip_952_view_3.jpg', 'slip_rgb/slip_952_view_4.jpg', 'slip_rgb/slip_952_view_5.jpg']
---------------------------------
['slip_rgb/slip_953_view_1.jpg', 'slip_rgb/slip_953_view_2.jpg', 'slip_rgb/slip_953_view_3.jpg', 'slip_rgb/slip_953_view_4.jpg', 'slip_rgb/slip_953_view_5.jpg']
---------------------------------
['slip_rgb/slip_954_view_1.jpg', 'slip_rgb/slip_954_view_2.jpg', 'slip_rgb/slip_954_view_3.jpg', 'slip_rgb/slip_954_view_4.jpg', 'slip_rgb/slip_954_view_5.jpg']
---------------------------------
['slip_rgb/slip_955_view_1.jpg', 'slip_rgb/slip_955_view_2.jpg', 'slip_rgb/slip_955_view_3.jpg', 'slip_rgb/slip_955_view_4.jpg', 'slip_rgb/slip_955_view_5.jpg']
-------------------------

---------------------------------
['slip_rgb/slip_994_view_1.jpg', 'slip_rgb/slip_994_view_2.jpg', 'slip_rgb/slip_994_view_3.jpg', 'slip_rgb/slip_994_view_4.jpg', 'slip_rgb/slip_994_view_5.jpg']
---------------------------------
['slip_rgb/slip_995_view_1.jpg', 'slip_rgb/slip_995_view_2.jpg', 'slip_rgb/slip_995_view_3.jpg', 'slip_rgb/slip_995_view_4.jpg', 'slip_rgb/slip_995_view_5.jpg']
---------------------------------
['slip_rgb/slip_996_view_1.jpg', 'slip_rgb/slip_996_view_2.jpg', 'slip_rgb/slip_996_view_3.jpg', 'slip_rgb/slip_996_view_4.jpg', 'slip_rgb/slip_996_view_5.jpg']
---------------------------------
['slip_rgb/slip_997_view_1.jpg', 'slip_rgb/slip_997_view_2.jpg', 'slip_rgb/slip_997_view_3.jpg', 'slip_rgb/slip_997_view_4.jpg', 'slip_rgb/slip_997_view_5.jpg']
---------------------------------
['slip_rgb/slip_998_view_1.jpg', 'slip_rgb/slip_998_view_2.jpg', 'slip_rgb/slip_998_view_3.jpg', 'slip_rgb/slip_998_view_4.jpg', 'slip_rgb/slip_998_view_5.jpg']
-------------------------

# Lace Type Shoes :

In [12]:
images_no = []
images = []
directory = listdir('lace_data_rgb/')
kmeans = joblib.load('kmeans_10_resnet_imagenet_laceup')

def cmp_dist(best_img,idx,result,k):
    
    if best_img[idx] == 0:
        best_img[idx] = {k:result}
    else :
        if list(best_img[idx].items())[0][1]< result :
            best_img[idx] = {k:result}       
        else:
            pass
    return best_img

def write_image(arr,idx,i):
    
    key = list(arr[idx].items())[0][0]
    img_path = key
    img_path = img_path.split('/')[-1]
#     img = cv2.imread('lace_data_rgb/'+img_path)
    img_path = img_path[:4]+'_up'+img_path[4:]
    img = cv2.imread('data/'+img_path)
    img = cv2.resize(img , (512,512))
    cv2.imwrite('Best_views/Laceup/'+i+'_'+'.jpg',img)


for i in directory :
    item = i.split('_')
    images_no.append(item[1])
    
    if images_no[-1] in images :
        pass
    else:
        images.append(item[0]+'_'+item[1])

        
for i in images :
    print (i)
    images_cmp = []
    # best_img = 0 * cluster size
    best_img = [0]*11
    for j in ['view_1','view_2','view_3','view_4','view_5']:
        image_path = i+'_'+j+'.jpg'
        
        if image_path in directory :
            images_cmp.append('lace_data_rgb/'+image_path)
#     print (images_cmp)        
    for k in images_cmp :
        img = image.load_img(k, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)
        resnet50_feature = model.predict(img_data)
        resnet50_feature = np.array(resnet50_feature.flatten())
        resnet50_feature = np.array(resnet50_feature)
        resnet50_feature = np.expand_dims(resnet50_feature, axis=0)
        #     print (resnet50_feature.shape)
        pred_view = kmeans.predict(resnet50_feature)
        k_index = int(k.split('_')[-1].split('.')[0]) - 1
#         print (pred_view)
        if pred_view[0] == 6 :
            result = spatial.distance.cosine(kmeans.cluster_centers_[6],resnet50_feature.flatten())
            best_img = cmp_dist(best_img,pred_view[0],result,k)
            
            
        if pred_view[0] == 2 :
            result = spatial.distance.cosine(kmeans.cluster_centers_[2],resnet50_feature.flatten())
            best_img = cmp_dist(best_img,pred_view[0],result,k)
            
        
            

    for l in best_img :
        if best_img[6] != 0:
            write_image(best_img,6,i)
            break
        
        elif best_img[2] !=0 :
            write_image(best_img,2,i)
            break
            
        else :
            print ("No Best Image")
            



lace_1260
lace_1260
lace_1260
lace_1260
lace_1260
lace_1261
lace_1261
lace_1261
lace_1261
lace_1261
lace_1262
lace_1262
lace_1262
lace_1262
lace_1262
lace_1263
lace_1263
lace_1263
lace_1263
lace_1263
lace_1264
lace_1264
lace_1264
lace_1264
lace_1264
lace_1265
lace_1265
lace_1265
lace_1265
lace_1265
lace_1266
lace_1266
lace_1266
lace_1266
lace_1266
lace_1267
lace_1267
lace_1267
lace_1267
lace_1267
lace_1268
lace_1268
lace_1268
lace_1268
lace_1268
lace_1269
lace_1269
lace_1269
lace_1269
lace_1269
lace_1270
lace_1270
lace_1270
lace_1270
lace_1270
lace_1271
lace_1271
lace_1271
lace_1271
lace_1271
lace_1272
lace_1272
lace_1272
lace_1272
lace_1272
lace_1273
lace_1273
lace_1273
lace_1273
lace_1273
lace_1274
lace_1274
lace_1274
lace_1274
lace_1274
lace_1275
lace_1275
lace_1275
lace_1275
lace_1275
lace_1276
lace_1276
lace_1276
lace_1276
lace_1276
lace_1277
lace_1277
lace_1277
lace_1277
lace_1277
lace_1278
lace_1278
lace_1278
lace_1278
lace_1278
lace_1279
No Best Image
No Best Image
No Best Imag

lace_1362
lace_1362
lace_1362
lace_1363
lace_1363
lace_1363
lace_1363
lace_1363
lace_1364
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
lace_1364
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
lace_1364
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
lace_1364
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
lace_1364
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
lace_1365
lace_1365
lace_1365
lace_1365
lace_1365
lace_1366
lace_1366
lace_1366
lace_1366
lace_1366


lace_1434
lace_1434
lace_1434
lace_1434
lace_1435
lace_1435
lace_1435
lace_1435
lace_1435
lace_1436
lace_1436
lace_1436
lace_1436
lace_1436
lace_1437
lace_1437
lace_1437
lace_1437
lace_1437
lace_1438
lace_1438
lace_1438
lace_1438
lace_1438
lace_1439
lace_1439
lace_1439
lace_1439
lace_1439
lace_1440
lace_1440
lace_1440
lace_1440
lace_1440
lace_1441
lace_1441
lace_1441
lace_1441
lace_1441
lace_1442
lace_1442
lace_1442
lace_1442
lace_1442
lace_1443
lace_1443
lace_1443
lace_1443
lace_1443
lace_1444
lace_1444
lace_1444
lace_1444
lace_1444
lace_1445
lace_1445
lace_1445
lace_1445
lace_1445
lace_1446
lace_1446
lace_1446
lace_1446
lace_1446
lace_1447
lace_1447
lace_1447
lace_1447
lace_1447
lace_1448
lace_1448
lace_1448
lace_1448
lace_1448
lace_1449
lace_1449
lace_1449
lace_1449
lace_1449
lace_1450
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
lace_1450
No Best Image
No Best Image
No Best 

lace_1562
lace_1562
lace_1562
lace_1562
lace_1562
lace_1563
lace_1563
lace_1563
lace_1563
lace_1563
lace_1564
lace_1564
lace_1564
lace_1564
lace_1564
lace_1565
lace_1565
lace_1565
lace_1565
lace_1565
lace_1566
lace_1566
lace_1566
lace_1566
lace_1566
lace_1567
lace_1567
lace_1567
lace_1567
lace_1567
lace_1568
lace_1568
lace_1568
lace_1568
lace_1568
lace_1569
lace_1569
lace_1569
lace_1569
lace_1569
lace_1570
lace_1570
lace_1570
lace_1570
lace_1570
lace_1571
lace_1571
lace_1571
lace_1571
lace_1571
lace_1572
lace_1572
lace_1572
lace_1572
lace_1572
lace_1573
lace_1573
lace_1573
lace_1573
lace_1573
lace_1574
lace_1574
lace_1574
lace_1574
lace_1574
lace_1575
lace_1575
lace_1575
lace_1575
lace_1575
lace_1576
lace_1576
lace_1576
lace_1576
lace_1576
lace_1577
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
lace_1577
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Bes

lace_1664
lace_1664
lace_1665
lace_1665
lace_1665
lace_1665
lace_1665
lace_1666
lace_1666
lace_1666
lace_1666
lace_1666
lace_1667
lace_1667
lace_1667
lace_1667
lace_1667
lace_1668
lace_1668
lace_1668
lace_1668
lace_1668
lace_1669
lace_1669
lace_1669
lace_1669
lace_1669
lace_1670
lace_1670
lace_1670
lace_1670
lace_1670
lace_1671
lace_1671
lace_1671
lace_1671
lace_1671
lace_1672
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
lace_1672
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
lace_1672
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
lace_1672
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No Best Image
No

In [51]:
images_no = [-1]
images = []
directory = listdir('lace_up_rgb/')
directory.pop(0)
kmeans = joblib.load('kmeans_clustering/kmeans_10_resnet_imagenet_laceup')


    
def write_image(best_img,cluster_no):
    key = ['temp']
    value = ['temp']
    data_directory = listdir('data')
    for i in best_img[1:] :
        if type(i)==dict:
            key.append( list(i.keys())[0] )
            value.append( list(i.values())[0] )
        else:
            key.append('temp')
            value.append('temp')
            
    if key.count(cluster_no)>1 : 
        image_name = best_img[0]
        dict_index = [0]
        for i in best_img[1:] :
            if type(i)==dict:
                if (list(i.keys())[0] ==cluster_no) :
                    if dict_index[0] == 0 :
                        index_no = best_img.index(i)
                        value_distance = list( i.values() )[0]
                        dict_index[0] = {index_no:value_distance}
                    
                    else :
                        index_no = best_img.index(i)
                        value_distance = list( i.values() )[0]
                        if list(dict_index[0].values())[0] > value_distance :
                            dict_index[0]={index_no:value_distance}
                                            
        best_image_view = str( list( dict_index[0].keys() )[0] )    
        best_image_name = image_name+'_view_'+best_image_view+'.jpg'
        img = cv2.imread('data/'+best_image_name)
        img = cv2.resize(img , (512,512))
        cv2.imwrite('Best_views/Laceup_best/'+best_image_name,img)
        for k in ['view_1','view_2','view_3','view_4','view_5']:
            full_image_name = image_name+'_'+k+'.jpg'
            if (full_image_name in data_directory and full_image_name!=best_image_name):
                img = cv2.imread('data/'+full_image_name)
                img = cv2.resize(img , (512,512))
                cv2.imwrite('Best_views/Laceup_nobest/'+image_name+'_'+k+'.jpg',img)
                
    else :
        image_name = best_img[0]
        dict_index = [0]
        for i in best_img[1:]:
            if type(i) == dict :
                if (list(i.keys())[0] ==cluster_no) :
                    index_no = best_img.index(i)
                    value_distance = list( i.values() )[0]
                    dict_index[0] = {index_no:value_distance}
                    
#         print ("-========",dict_index)
        best_image_view = str( list( dict_index[0].keys() )[0] )    
        best_image_name = image_name+'_view_'+best_image_view+'.jpg'
        img = cv2.imread('data/'+best_image_name)
        img = cv2.resize(img , (512,512))
        cv2.imwrite('Best_views/Laceup_best/'+best_image_name,img)
        for k in ['view_1','view_2','view_3','view_4','view_5']:
            full_image_name = image_name+'_'+k+'.jpg'
            if (full_image_name in data_directory and full_image_name!=best_image_name):
                img = cv2.imread('data/'+full_image_name)
                img = cv2.resize(img , (512,512))
                cv2.imwrite('Best_views/Laceup_nobest/'+image_name+'_'+k+'.jpg',img)
                
            
def write_all_image(best_img):
    data_directory = listdir('data')
    image_name = best_img[0]
    for k in ['view_1','view_2','view_3','view_4','view_5']:
        full_image_name = image_name+'_'+k+'.jpg'
        if (full_image_name in data_directory ):
            img = cv2.imread('data/'+full_image_name)
            img = cv2.resize(img , (512,512))
            cv2.imwrite('Best_views/Laceup_nobest/'+image_name+'_'+k+'.jpg',img)
    

    
def same_image(images):
    images_cmp = []
    best_img = [0]*6
    best_img[0]=images
    for j in ['view_1','view_2','view_3','view_4','view_5']:
        image_path = images+'_'+j+'.jpg'

        if image_path in directory :
            images_cmp.append('lace_up_rgb/'+image_path)
                
    return images_cmp,best_img


for i in directory :
    item = i.split('_')
    if item[1] == images_no[-1]:
        pass
    else :
        images_no.append(item[1])
        images.append(item[0]+'_'+item[1])


 
    
for k in range(len(images)) :
    print ("---------------------------------")
    images_cmp,best_img = same_image(images[k])
    print (images_cmp)
    for l in images_cmp :
        img = image.load_img(l, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)
        resnet50_feature = model.predict(img_data)
        resnet50_feature = np.array(resnet50_feature.flatten())
        resnet50_feature = np.array(resnet50_feature)
        resnet50_feature = np.expand_dims(resnet50_feature, axis=0)
        #     print (resnet50_feature.shape)
        pred_view = kmeans.predict(resnet50_feature)
        l_index = int(l.split('_')[-1].split('.')[0])

        if pred_view[0] in [6,2]:
            result = spatial.distance.cosine(kmeans.cluster_centers_[pred_view[0]],resnet50_feature.flatten())
            best_img[l_index] = {pred_view[0]:result}
            

        else :
            result = spatial.distance.cosine(kmeans.cluster_centers_[pred_view[0]],resnet50_feature.flatten())
            best_img[l_index] = {pred_view[0]:result}
            
    
            
    key =[]
    for m in range(1,len(best_img)):

        if (type(best_img[m]) == dict):
            key.append( list(best_img[m].keys())[0] )
            
            
    if 6 in key :
        write_image(best_img,6)
        continue
        
    elif 2 in key :
        write_image(best_img,2)
        continue
        
    else :
        print ('here')
        write_all_image(best_img)
        continue
        
    
    


---------------------------------
['lace_up_rgb/lace_1260_view_2.jpg', 'lace_up_rgb/lace_1260_view_3.jpg', 'lace_up_rgb/lace_1260_view_4.jpg', 'lace_up_rgb/lace_1260_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1261_view_1.jpg', 'lace_up_rgb/lace_1261_view_2.jpg', 'lace_up_rgb/lace_1261_view_3.jpg', 'lace_up_rgb/lace_1261_view_4.jpg', 'lace_up_rgb/lace_1261_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1262_view_1.jpg', 'lace_up_rgb/lace_1262_view_2.jpg', 'lace_up_rgb/lace_1262_view_3.jpg', 'lace_up_rgb/lace_1262_view_4.jpg', 'lace_up_rgb/lace_1262_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1263_view_1.jpg', 'lace_up_rgb/lace_1263_view_2.jpg', 'lace_up_rgb/lace_1263_view_3.jpg', 'lace_up_rgb/lace_1263_view_4.jpg', 'lace_up_rgb/lace_1263_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1264_view_1.jpg', 'lace_up_rgb/lace_1264_view_2.jpg', 'lace_up_rgb/lace_1264_view_3.jpg', 'lace_up_rgb/lace_1264_view_4.jpg

---------------------------------
['lace_up_rgb/lace_1299_view_1.jpg', 'lace_up_rgb/lace_1299_view_2.jpg', 'lace_up_rgb/lace_1299_view_3.jpg', 'lace_up_rgb/lace_1299_view_4.jpg', 'lace_up_rgb/lace_1299_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1300_view_1.jpg', 'lace_up_rgb/lace_1300_view_2.jpg', 'lace_up_rgb/lace_1300_view_3.jpg', 'lace_up_rgb/lace_1300_view_4.jpg', 'lace_up_rgb/lace_1300_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1301_view_1.jpg', 'lace_up_rgb/lace_1301_view_2.jpg', 'lace_up_rgb/lace_1301_view_3.jpg', 'lace_up_rgb/lace_1301_view_4.jpg', 'lace_up_rgb/lace_1301_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1302_view_1.jpg', 'lace_up_rgb/lace_1302_view_2.jpg', 'lace_up_rgb/lace_1302_view_3.jpg', 'lace_up_rgb/lace_1302_view_4.jpg', 'lace_up_rgb/lace_1302_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1303_view_1.jpg', 'lace_up_rgb/lace_1303_view_2.jpg', 'lace_up_rgb/lace_1303_view_3.jpg

---------------------------------
['lace_up_rgb/lace_1338_view_1.jpg', 'lace_up_rgb/lace_1338_view_2.jpg', 'lace_up_rgb/lace_1338_view_3.jpg', 'lace_up_rgb/lace_1338_view_4.jpg', 'lace_up_rgb/lace_1338_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1339_view_1.jpg', 'lace_up_rgb/lace_1339_view_2.jpg', 'lace_up_rgb/lace_1339_view_3.jpg', 'lace_up_rgb/lace_1339_view_4.jpg', 'lace_up_rgb/lace_1339_view_5.jpg']
here
---------------------------------
['lace_up_rgb/lace_1340_view_1.jpg', 'lace_up_rgb/lace_1340_view_2.jpg', 'lace_up_rgb/lace_1340_view_3.jpg', 'lace_up_rgb/lace_1340_view_4.jpg', 'lace_up_rgb/lace_1340_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1341_view_1.jpg', 'lace_up_rgb/lace_1341_view_2.jpg', 'lace_up_rgb/lace_1341_view_3.jpg', 'lace_up_rgb/lace_1341_view_4.jpg', 'lace_up_rgb/lace_1341_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1342_view_1.jpg', 'lace_up_rgb/lace_1342_view_2.jpg', 'lace_up_rgb/lace_1342_view_

---------------------------------
['lace_up_rgb/lace_1377_view_1.jpg', 'lace_up_rgb/lace_1377_view_2.jpg', 'lace_up_rgb/lace_1377_view_3.jpg', 'lace_up_rgb/lace_1377_view_4.jpg', 'lace_up_rgb/lace_1377_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1378_view_1.jpg', 'lace_up_rgb/lace_1378_view_2.jpg', 'lace_up_rgb/lace_1378_view_3.jpg', 'lace_up_rgb/lace_1378_view_4.jpg', 'lace_up_rgb/lace_1378_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1379_view_1.jpg', 'lace_up_rgb/lace_1379_view_2.jpg', 'lace_up_rgb/lace_1379_view_3.jpg', 'lace_up_rgb/lace_1379_view_4.jpg', 'lace_up_rgb/lace_1379_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1380_view_1.jpg', 'lace_up_rgb/lace_1380_view_2.jpg', 'lace_up_rgb/lace_1380_view_3.jpg', 'lace_up_rgb/lace_1380_view_4.jpg', 'lace_up_rgb/lace_1380_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1381_view_1.jpg', 'lace_up_rgb/lace_1381_view_2.jpg', 'lace_up_rgb/lace_1381_view_3.jpg

---------------------------------
['lace_up_rgb/lace_1416_view_1.jpg', 'lace_up_rgb/lace_1416_view_2.jpg', 'lace_up_rgb/lace_1416_view_3.jpg', 'lace_up_rgb/lace_1416_view_4.jpg', 'lace_up_rgb/lace_1416_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1417_view_1.jpg', 'lace_up_rgb/lace_1417_view_2.jpg', 'lace_up_rgb/lace_1417_view_3.jpg', 'lace_up_rgb/lace_1417_view_4.jpg', 'lace_up_rgb/lace_1417_view_5.jpg']
here
---------------------------------
['lace_up_rgb/lace_1418_view_1.jpg', 'lace_up_rgb/lace_1418_view_2.jpg', 'lace_up_rgb/lace_1418_view_3.jpg', 'lace_up_rgb/lace_1418_view_4.jpg', 'lace_up_rgb/lace_1418_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1419_view_1.jpg', 'lace_up_rgb/lace_1419_view_2.jpg', 'lace_up_rgb/lace_1419_view_3.jpg', 'lace_up_rgb/lace_1419_view_4.jpg', 'lace_up_rgb/lace_1419_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1420_view_1.jpg', 'lace_up_rgb/lace_1420_view_2.jpg', 'lace_up_rgb/lace_1420_view_

...

---------------------------------
['lace_up_rgb/lace_1533_view_1.jpg', 'lace_up_rgb/lace_1533_view_2.jpg', 'lace_up_rgb/lace_1533_view_3.jpg', 'lace_up_rgb/lace_1533_view_4.jpg', 'lace_up_rgb/lace_1533_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1534_view_1.jpg', 'lace_up_rgb/lace_1534_view_2.jpg', 'lace_up_rgb/lace_1534_view_3.jpg', 'lace_up_rgb/lace_1534_view_4.jpg', 'lace_up_rgb/lace_1534_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1535_view_1.jpg', 'lace_up_rgb/lace_1535_view_2.jpg', 'lace_up_rgb/lace_1535_view_3.jpg', 'lace_up_rgb/lace_1535_view_4.jpg', 'lace_up_rgb/lace_1535_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1536_view_1.jpg', 'lace_up_rgb/lace_1536_view_2.jpg', 'lace_up_rgb/lace_1536_view_3.jpg', 'lace_up_rgb/lace_1536_view_4.jpg', 'lace_up_rgb/lace_1536_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1537_view_1.jpg', 'lace_up_rgb/lace_1537_view_2.jpg', 'lace_up_rgb/lace_1537_view_3.jpg

---------------------------------
['lace_up_rgb/lace_1572_view_1.jpg', 'lace_up_rgb/lace_1572_view_2.jpg', 'lace_up_rgb/lace_1572_view_3.jpg', 'lace_up_rgb/lace_1572_view_4.jpg', 'lace_up_rgb/lace_1572_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1573_view_1.jpg', 'lace_up_rgb/lace_1573_view_2.jpg', 'lace_up_rgb/lace_1573_view_3.jpg', 'lace_up_rgb/lace_1573_view_4.jpg', 'lace_up_rgb/lace_1573_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1574_view_1.jpg', 'lace_up_rgb/lace_1574_view_2.jpg', 'lace_up_rgb/lace_1574_view_3.jpg', 'lace_up_rgb/lace_1574_view_4.jpg', 'lace_up_rgb/lace_1574_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1575_view_1.jpg', 'lace_up_rgb/lace_1575_view_2.jpg', 'lace_up_rgb/lace_1575_view_3.jpg', 'lace_up_rgb/lace_1575_view_4.jpg', 'lace_up_rgb/lace_1575_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1576_view_1.jpg', 'lace_up_rgb/lace_1576_view_2.jpg', 'lace_up_rgb/lace_1576_view_3.jpg

here
---------------------------------
['lace_up_rgb/lace_1611_view_1.jpg', 'lace_up_rgb/lace_1611_view_2.jpg', 'lace_up_rgb/lace_1611_view_3.jpg', 'lace_up_rgb/lace_1611_view_4.jpg', 'lace_up_rgb/lace_1611_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1612_view_1.jpg', 'lace_up_rgb/lace_1612_view_2.jpg', 'lace_up_rgb/lace_1612_view_3.jpg', 'lace_up_rgb/lace_1612_view_4.jpg', 'lace_up_rgb/lace_1612_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1613_view_1.jpg', 'lace_up_rgb/lace_1613_view_2.jpg', 'lace_up_rgb/lace_1613_view_3.jpg', 'lace_up_rgb/lace_1613_view_4.jpg', 'lace_up_rgb/lace_1613_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1614_view_1.jpg', 'lace_up_rgb/lace_1614_view_2.jpg', 'lace_up_rgb/lace_1614_view_3.jpg', 'lace_up_rgb/lace_1614_view_4.jpg', 'lace_up_rgb/lace_1614_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1615_view_1.jpg', 'lace_up_rgb/lace_1615_view_2.jpg', 'lace_up_rgb/lace_1615_view_

---------------------------------
['lace_up_rgb/lace_1650_view_1.jpg', 'lace_up_rgb/lace_1650_view_2.jpg', 'lace_up_rgb/lace_1650_view_3.jpg', 'lace_up_rgb/lace_1650_view_4.jpg', 'lace_up_rgb/lace_1650_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1651_view_1.jpg', 'lace_up_rgb/lace_1651_view_2.jpg', 'lace_up_rgb/lace_1651_view_3.jpg', 'lace_up_rgb/lace_1651_view_4.jpg', 'lace_up_rgb/lace_1651_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1652_view_1.jpg', 'lace_up_rgb/lace_1652_view_2.jpg', 'lace_up_rgb/lace_1652_view_3.jpg', 'lace_up_rgb/lace_1652_view_4.jpg', 'lace_up_rgb/lace_1652_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1653_view_1.jpg', 'lace_up_rgb/lace_1653_view_2.jpg', 'lace_up_rgb/lace_1653_view_3.jpg', 'lace_up_rgb/lace_1653_view_4.jpg', 'lace_up_rgb/lace_1653_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1654_view_1.jpg', 'lace_up_rgb/lace_1654_view_2.jpg', 'lace_up_rgb/lace_1654_view_3.jpg

---------------------------------
['lace_up_rgb/lace_1689_view_1.jpg', 'lace_up_rgb/lace_1689_view_2.jpg', 'lace_up_rgb/lace_1689_view_3.jpg', 'lace_up_rgb/lace_1689_view_4.jpg', 'lace_up_rgb/lace_1689_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1690_view_1.jpg', 'lace_up_rgb/lace_1690_view_2.jpg', 'lace_up_rgb/lace_1690_view_3.jpg', 'lace_up_rgb/lace_1690_view_4.jpg', 'lace_up_rgb/lace_1690_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1691_view_1.jpg', 'lace_up_rgb/lace_1691_view_2.jpg', 'lace_up_rgb/lace_1691_view_3.jpg', 'lace_up_rgb/lace_1691_view_4.jpg', 'lace_up_rgb/lace_1691_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1692_view_1.jpg', 'lace_up_rgb/lace_1692_view_2.jpg', 'lace_up_rgb/lace_1692_view_3.jpg', 'lace_up_rgb/lace_1692_view_4.jpg', 'lace_up_rgb/lace_1692_view_5.jpg']
---------------------------------
['lace_up_rgb/lace_1693_view_1.jpg', 'lace_up_rgb/lace_1693_view_2.jpg', 'lace_up_rgb/lace_1693_view_3.jpg

# Backstrap Type Shoes :  

In [49]:
images_no = [0]
images = []
directory = listdir('backstrap_rgb/')
directory.pop(0)
kmeans = joblib.load('kmeans_clustering/kmeans_10_resnet_imagenet_backstrap')

def cmp_dist(best_img,idx,result,k):
    
    if best_img[idx] == 0:
        best_img[idx] = {k:result}
    else :
        if list(best_img[idx].items())[0][1]< result :
            best_img[idx] = {k:result}       
        else:
            pass
    return best_img

# def write_image(arr,idx,i):
    
#     key = list(arr[idx].items())[0][0]
#     img_path = key
#     img_path = img_path.split('/')[-1]
# #     img = cv2.imread('lace_data_rgb/'+img_path)
# #     img_path = img_path[:4]+'_up'+img_path[4:]
#     img = cv2.imread('data/'+img_path)
#     img = cv2.resize(img , (512,512))
#     cv2.imwrite('Best_views/Backstrap/'+i+'_'+'.jpg',img)
    
    
def write_image(best_img,cluster_no):
    key = ['temp']
    value = ['temp']
    data_directory = listdir('data')
    for i in best_img[1:] :
        if type(i)==dict:
            key.append( list(i.keys())[0] )
            value.append( list(i.values())[0] )
        else:
            key.append('temp')
            value.append('temp')
            
    if key.count(cluster_no)>1 : 
        image_name = best_img[0]
        dict_index = [0]
        for i in best_img[1:] :
            if type(i)==dict:
                if (list(i.keys())[0] ==cluster_no) :
                    if dict_index[0] == 0 :
                        index_no = best_img.index(i)
                        value_distance = list( i.values() )[0]
                        dict_index[0] = {index_no:value_distance}
                    
                    else :
                        index_no = best_img.index(i)
                        value_distance = list( i.values() )[0]
                        if list(dict_index[0].values())[0] > value_distance :
                            dict_index[0]={index_no:value_distance}
                                            
        best_image_view = str( list( dict_index[0].keys() )[0] )    
        best_image_name = image_name+'_view_'+best_image_view+'.jpg'
        img = cv2.imread('data/'+best_image_name)
        img = cv2.resize(img , (512,512))
        cv2.imwrite('Best_views/Backstrap_best/'+best_image_name,img)
        for k in ['view_1','view_2','view_3','view_4','view_5']:
            full_image_name = image_name+'_'+k+'.jpg'
            if (full_image_name in data_directory and full_image_name!=best_image_name):
                img = cv2.imread('data/'+full_image_name)
                img = cv2.resize(img , (512,512))
                cv2.imwrite('Best_views/Backstrap_notbest/'+image_name+'_'+k+'.jpg',img)
                
    else :
        image_name = best_img[0]
        dict_index = [0]
        for i in best_img[1:]:
            if type(i) == dict :
                if (list(i.keys())[0] ==cluster_no) :
                    index_no = best_img.index(i)
                    value_distance = list( i.values() )[0]
                    dict_index[0] = {index_no:value_distance}
                    
#         print ("-========",dict_index)
        best_image_view = str( list( dict_index[0].keys() )[0] )    
        best_image_name = image_name+'_view_'+best_image_view+'.jpg'
        img = cv2.imread('data/'+best_image_name)
        img = cv2.resize(img , (512,512))
        cv2.imwrite('Best_views/Backstrap_best/'+best_image_name,img)
        for k in ['view_1','view_2','view_3','view_4','view_5']:
            full_image_name = image_name+'_'+k+'.jpg'
            if (full_image_name in data_directory and full_image_name!=best_image_name):
                img = cv2.imread('data/'+full_image_name)
                img = cv2.resize(img , (512,512))
                cv2.imwrite('Best_views/Backstrap_notbest/'+image_name+'_'+k+'.jpg',img)
                
            
def write_all_image(best_img):
    data_directory = listdir('data')
    image_name = best_img[0]
    for k in ['view_1','view_2','view_3','view_4','view_5']:
        full_image_name = image_name+'_'+k+'.jpg'
        if (full_image_name in data_directory ):
            img = cv2.imread('data/'+full_image_name)
            img = cv2.resize(img , (512,512))
            cv2.imwrite('Best_views/Backstrap_notbest/'+image_name+'_'+k+'.jpg',img)
    

    
def same_image(images):
    images_cmp = []
    best_img = [0]*6
    best_img[0]=images
    for j in ['view_1','view_2','view_3','view_4','view_5']:
        image_path = images+'_'+j+'.jpg'

        if image_path in directory :
            images_cmp.append('backstrap_rgb/'+image_path)
                
    return images_cmp,best_img


for i in directory :
    item = i.split('_')
    if item[1] == images_no[-1]:
        pass
    else :
        images_no.append(item[1])
        images.append(item[0]+'_'+item[1])


 
    
for k in range(len(images)) :
    print ("---------------------------------")
    images_cmp,best_img = same_image(images[k])
    print (images_cmp)
    for l in images_cmp :
        img = image.load_img(l, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)
        resnet50_feature = model.predict(img_data)
        resnet50_feature = np.array(resnet50_feature.flatten())
        resnet50_feature = np.array(resnet50_feature)
        resnet50_feature = np.expand_dims(resnet50_feature, axis=0)
        #     print (resnet50_feature.shape)
        pred_view = kmeans.predict(resnet50_feature)
        l_index = int(l.split('_')[-1].split('.')[0])

        if pred_view[0] in [1,3,8,6]:
            result = spatial.distance.cosine(kmeans.cluster_centers_[pred_view[0]],resnet50_feature.flatten())
            best_img[l_index] = {pred_view[0]:result}
            

        else :
            result = spatial.distance.cosine(kmeans.cluster_centers_[pred_view[0]],resnet50_feature.flatten())
            best_img[l_index] = {pred_view[0]:result}
            
    
            
    key =[]
    for m in range(1,len(best_img)):

        if (type(best_img[m]) == dict):
            key.append( list(best_img[m].keys())[0] )
            
            
    if 1 in key :
        write_image(best_img,1)
        continue
        
    elif 3 in key :
        write_image(best_img,3)
        continue

    elif 8 in key :
        write_image(best_img,8)
        continue
        
    elif 6 in key :
        write_image(best_img,6)
        continue
        
    else :
        print ('here')
        write_all_image(best_img)
        continue
        
    
    


---------------------------------
['backstrap_rgb/backstrap_500_view_1.jpg', 'backstrap_rgb/backstrap_500_view_2.jpg', 'backstrap_rgb/backstrap_500_view_3.jpg', 'backstrap_rgb/backstrap_500_view_4.jpg', 'backstrap_rgb/backstrap_500_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_501_view_1.jpg', 'backstrap_rgb/backstrap_501_view_2.jpg', 'backstrap_rgb/backstrap_501_view_3.jpg', 'backstrap_rgb/backstrap_501_view_4.jpg', 'backstrap_rgb/backstrap_501_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_502_view_1.jpg', 'backstrap_rgb/backstrap_502_view_2.jpg', 'backstrap_rgb/backstrap_502_view_3.jpg', 'backstrap_rgb/backstrap_502_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_503_view_1.jpg', 'backstrap_rgb/backstrap_503_view_2.jpg', 'backstrap_rgb/backstrap_503_view_3.jpg', 'backstrap_rgb/backstrap_503_view_4.jpg', 'backstrap_rgb/backstrap_503_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_50

---------------------------------
['backstrap_rgb/backstrap_535_view_1.jpg', 'backstrap_rgb/backstrap_535_view_2.jpg', 'backstrap_rgb/backstrap_535_view_3.jpg', 'backstrap_rgb/backstrap_535_view_4.jpg', 'backstrap_rgb/backstrap_535_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_536_view_1.jpg', 'backstrap_rgb/backstrap_536_view_2.jpg', 'backstrap_rgb/backstrap_536_view_3.jpg', 'backstrap_rgb/backstrap_536_view_4.jpg', 'backstrap_rgb/backstrap_536_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_537_view_1.jpg', 'backstrap_rgb/backstrap_537_view_2.jpg', 'backstrap_rgb/backstrap_537_view_3.jpg', 'backstrap_rgb/backstrap_537_view_4.jpg', 'backstrap_rgb/backstrap_537_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_538_view_1.jpg', 'backstrap_rgb/backstrap_538_view_2.jpg', 'backstrap_rgb/backstrap_538_view_3.jpg', 'backstrap_rgb/backstrap_538_view_4.jpg', 'backstrap_rgb/backstrap_538_view_5.jpg']
--------------------

---------------------------------
['backstrap_rgb/backstrap_569_view_1.jpg', 'backstrap_rgb/backstrap_569_view_2.jpg', 'backstrap_rgb/backstrap_569_view_3.jpg', 'backstrap_rgb/backstrap_569_view_4.jpg', 'backstrap_rgb/backstrap_569_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_570_view_1.jpg', 'backstrap_rgb/backstrap_570_view_2.jpg', 'backstrap_rgb/backstrap_570_view_3.jpg', 'backstrap_rgb/backstrap_570_view_4.jpg', 'backstrap_rgb/backstrap_570_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_571_view_1.jpg', 'backstrap_rgb/backstrap_571_view_2.jpg', 'backstrap_rgb/backstrap_571_view_3.jpg', 'backstrap_rgb/backstrap_571_view_4.jpg', 'backstrap_rgb/backstrap_571_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_572_view_1.jpg', 'backstrap_rgb/backstrap_572_view_2.jpg', 'backstrap_rgb/backstrap_572_view_3.jpg', 'backstrap_rgb/backstrap_572_view_4.jpg', 'backstrap_rgb/backstrap_572_view_5.jpg']
--------------------

---------------------------------
['backstrap_rgb/backstrap_603_view_1.jpg', 'backstrap_rgb/backstrap_603_view_2.jpg', 'backstrap_rgb/backstrap_603_view_3.jpg', 'backstrap_rgb/backstrap_603_view_4.jpg', 'backstrap_rgb/backstrap_603_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_604_view_1.jpg', 'backstrap_rgb/backstrap_604_view_2.jpg', 'backstrap_rgb/backstrap_604_view_3.jpg', 'backstrap_rgb/backstrap_604_view_4.jpg', 'backstrap_rgb/backstrap_604_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_605_view_1.jpg', 'backstrap_rgb/backstrap_605_view_2.jpg', 'backstrap_rgb/backstrap_605_view_3.jpg', 'backstrap_rgb/backstrap_605_view_4.jpg', 'backstrap_rgb/backstrap_605_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_606_view_1.jpg', 'backstrap_rgb/backstrap_606_view_2.jpg', 'backstrap_rgb/backstrap_606_view_3.jpg', 'backstrap_rgb/backstrap_606_view_4.jpg', 'backstrap_rgb/backstrap_606_view_5.jpg']
--------------------

---------------------------------
['backstrap_rgb/backstrap_637_view_1.jpg', 'backstrap_rgb/backstrap_637_view_2.jpg', 'backstrap_rgb/backstrap_637_view_3.jpg', 'backstrap_rgb/backstrap_637_view_4.jpg', 'backstrap_rgb/backstrap_637_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_638_view_1.jpg', 'backstrap_rgb/backstrap_638_view_2.jpg', 'backstrap_rgb/backstrap_638_view_3.jpg', 'backstrap_rgb/backstrap_638_view_4.jpg', 'backstrap_rgb/backstrap_638_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_639_view_1.jpg', 'backstrap_rgb/backstrap_639_view_2.jpg', 'backstrap_rgb/backstrap_639_view_3.jpg', 'backstrap_rgb/backstrap_639_view_4.jpg', 'backstrap_rgb/backstrap_639_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_640_view_1.jpg', 'backstrap_rgb/backstrap_640_view_2.jpg', 'backstrap_rgb/backstrap_640_view_3.jpg', 'backstrap_rgb/backstrap_640_view_4.jpg', 'backstrap_rgb/backstrap_640_view_5.jpg']
--------------------

---------------------------------
['backstrap_rgb/backstrap_671_view_1.jpg', 'backstrap_rgb/backstrap_671_view_2.jpg', 'backstrap_rgb/backstrap_671_view_3.jpg', 'backstrap_rgb/backstrap_671_view_4.jpg', 'backstrap_rgb/backstrap_671_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_672_view_1.jpg', 'backstrap_rgb/backstrap_672_view_2.jpg', 'backstrap_rgb/backstrap_672_view_3.jpg', 'backstrap_rgb/backstrap_672_view_4.jpg', 'backstrap_rgb/backstrap_672_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_673_view_1.jpg', 'backstrap_rgb/backstrap_673_view_2.jpg', 'backstrap_rgb/backstrap_673_view_3.jpg', 'backstrap_rgb/backstrap_673_view_4.jpg', 'backstrap_rgb/backstrap_673_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_674_view_1.jpg', 'backstrap_rgb/backstrap_674_view_2.jpg', 'backstrap_rgb/backstrap_674_view_3.jpg', 'backstrap_rgb/backstrap_674_view_4.jpg', 'backstrap_rgb/backstrap_674_view_5.jpg']
--------------------

---------------------------------
['backstrap_rgb/backstrap_705_view_1.jpg', 'backstrap_rgb/backstrap_705_view_2.jpg', 'backstrap_rgb/backstrap_705_view_3.jpg', 'backstrap_rgb/backstrap_705_view_4.jpg', 'backstrap_rgb/backstrap_705_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_706_view_1.jpg', 'backstrap_rgb/backstrap_706_view_2.jpg', 'backstrap_rgb/backstrap_706_view_3.jpg', 'backstrap_rgb/backstrap_706_view_4.jpg', 'backstrap_rgb/backstrap_706_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_707_view_1.jpg', 'backstrap_rgb/backstrap_707_view_2.jpg', 'backstrap_rgb/backstrap_707_view_3.jpg', 'backstrap_rgb/backstrap_707_view_4.jpg', 'backstrap_rgb/backstrap_707_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_708_view_1.jpg', 'backstrap_rgb/backstrap_708_view_2.jpg', 'backstrap_rgb/backstrap_708_view_3.jpg', 'backstrap_rgb/backstrap_708_view_4.jpg', 'backstrap_rgb/backstrap_708_view_5.jpg']
--------------------

---------------------------------
['backstrap_rgb/backstrap_739_view_1.jpg', 'backstrap_rgb/backstrap_739_view_2.jpg', 'backstrap_rgb/backstrap_739_view_3.jpg', 'backstrap_rgb/backstrap_739_view_4.jpg', 'backstrap_rgb/backstrap_739_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_740_view_1.jpg', 'backstrap_rgb/backstrap_740_view_2.jpg', 'backstrap_rgb/backstrap_740_view_3.jpg', 'backstrap_rgb/backstrap_740_view_4.jpg', 'backstrap_rgb/backstrap_740_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_741_view_1.jpg', 'backstrap_rgb/backstrap_741_view_2.jpg', 'backstrap_rgb/backstrap_741_view_3.jpg', 'backstrap_rgb/backstrap_741_view_4.jpg', 'backstrap_rgb/backstrap_741_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_742_view_1.jpg', 'backstrap_rgb/backstrap_742_view_2.jpg', 'backstrap_rgb/backstrap_742_view_3.jpg', 'backstrap_rgb/backstrap_742_view_4.jpg', 'backstrap_rgb/backstrap_742_view_5.jpg']
--------------------

---------------------------------
['backstrap_rgb/backstrap_773_view_1.jpg', 'backstrap_rgb/backstrap_773_view_2.jpg', 'backstrap_rgb/backstrap_773_view_3.jpg', 'backstrap_rgb/backstrap_773_view_4.jpg', 'backstrap_rgb/backstrap_773_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_774_view_1.jpg', 'backstrap_rgb/backstrap_774_view_2.jpg', 'backstrap_rgb/backstrap_774_view_3.jpg', 'backstrap_rgb/backstrap_774_view_4.jpg', 'backstrap_rgb/backstrap_774_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_775_view_1.jpg', 'backstrap_rgb/backstrap_775_view_2.jpg', 'backstrap_rgb/backstrap_775_view_3.jpg', 'backstrap_rgb/backstrap_775_view_4.jpg', 'backstrap_rgb/backstrap_775_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_776_view_1.jpg', 'backstrap_rgb/backstrap_776_view_2.jpg', 'backstrap_rgb/backstrap_776_view_3.jpg', 'backstrap_rgb/backstrap_776_view_4.jpg', 'backstrap_rgb/backstrap_776_view_5.jpg']
--------------------

---------------------------------
['backstrap_rgb/backstrap_808_view_1.jpg', 'backstrap_rgb/backstrap_808_view_2.jpg', 'backstrap_rgb/backstrap_808_view_3.jpg', 'backstrap_rgb/backstrap_808_view_4.jpg', 'backstrap_rgb/backstrap_808_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_809_view_1.jpg', 'backstrap_rgb/backstrap_809_view_2.jpg', 'backstrap_rgb/backstrap_809_view_3.jpg', 'backstrap_rgb/backstrap_809_view_4.jpg', 'backstrap_rgb/backstrap_809_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_810_view_1.jpg', 'backstrap_rgb/backstrap_810_view_2.jpg', 'backstrap_rgb/backstrap_810_view_3.jpg', 'backstrap_rgb/backstrap_810_view_4.jpg', 'backstrap_rgb/backstrap_810_view_5.jpg']
---------------------------------
['backstrap_rgb/backstrap_811_view_1.jpg', 'backstrap_rgb/backstrap_811_view_2.jpg', 'backstrap_rgb/backstrap_811_view_3.jpg', 'backstrap_rgb/backstrap_811_view_4.jpg', 'backstrap_rgb/backstrap_811_view_5.jpg']
--------------------